In [ ]:
# Run this in Google Colab at your project root (so ./data/raw/ exists)
# It will print a plain-text summary for each raw data file under ./data/raw/
# Copy the printed output and paste it back to me.

import os
import json
import pandas as pd
import numpy as np
from collections import Counter
from pathlib import Path

ROOT = Path("/content/drive/MyDrive/player_value_prediction_project/data/raw")   # <- matches the structure you shared

# tune these for performance
CHUNK_SIZE = 200_000
MAX_TOP_UNIQUE_TO_SHOW = 10   # for categorical value_counts
MAX_ROWS_HEAD = 5

def safe_row_count_csv(path):
    """Count rows for CSV quickly (may be faster than pandas for large files)."""
    try:
        with open(path, "rb") as f:
            # count newline bytes; works well for typical CSVs
            count = 0
            buf = f.raw.read(1024*1024)
            while buf:
                count += buf.count(b"\n")
                buf = f.raw.read(1024*1024)
        # subtract header line if >0
        return max(0, count - 1)
    except Exception:
        # fallback to pandas (may be slow)
        try:
            return sum(1 for _ in pd.read_csv(path, chunksize=CHUNK_SIZE)) * CHUNK_SIZE
        except Exception:
            return None

def summarize_csv(path):
    path = Path(path)
    print("="*120)
    print(f"CSV: {path}")
    try:
        nrows = safe_row_count_csv(path)
    except Exception:
        nrows = None
    print(f"Estimated rows (header excluded): {nrows}")
    # read a small sample/head + dtypes via pandas
    try:
        df_head = pd.read_csv(path, nrows=MAX_ROWS_HEAD)
        print(f"Columns ({len(df_head.columns)}): {list(df_head.columns)}")
        print("\nHead:")
        print(df_head.head(MAX_ROWS_HEAD).to_string(index=False))
        # dtypes snapshot
        dtypes = df_head.dtypes.apply(lambda x: str(x)).to_dict()
        print("\nColumn dtypes (sample-based):")
        for c,t in dtypes.items():
            print(f"  - {c}: {t}")
    except Exception as e:
        print(f"Could not read HEAD/dtypes: {e}")
        return

    # compute missingness and basic stats in chunks to avoid large memory use
    try:
        missing_counts = Counter()
        nonnull_counts = Counter()
        numeric_stats = {}
        value_counts_samples = {}
        total_rows = 0
        # collect candidate categorical columns from the head
        candidate_cats = [c for c,dt in df_head.dtypes.items() if dt == "object" or dt == "string"]
        numeric_cols = [c for c,dt in df_head.dtypes.items() if np.issubdtype(df_head[c].dtype, np.number)]
        # We'll keep track of unique value samples for object cols (but not full unique sets)
        sample_unique_vals = {c: Counter() for c in candidate_cats}

        for chunk in pd.read_csv(path, chunksize=CHUNK_SIZE, low_memory=False):
            total_rows += len(chunk)
            for col in chunk.columns:
                missing_counts[col] += chunk[col].isna().sum()
                nonnull_counts[col] += chunk[col].notna().sum()

            # numeric stats incremental (mean/std approx via Welford)
            for col in chunk.select_dtypes(include=[np.number]).columns:
                arr = chunk[col].dropna().to_numpy()
                if arr.size == 0:
                    continue
                if col not in numeric_stats:
                    numeric_stats[col] = {"count":0, "sum":0.0, "sum_sq":0.0, "min":float("inf"), "max":float("-inf")}
                s = numeric_stats[col]
                s["count"] += arr.size
                s["sum"] += float(arr.sum())
                s["sum_sq"] += float((arr**2).sum())
                s["min"] = min(s["min"], float(arr.min()))
                s["max"] = max(s["max"], float(arr.max()))

            # sample value counts for categorical columns
            for col in candidate_cats:
                vals = chunk[col].dropna().astype(str)
                sample_unique_vals[col].update(vals.value_counts().to_dict())

        # summarize results
        print(f"\nProcessed rows (counted via chunks): {total_rows}  (Note: this may be < estimated rows if header parsing differed)")
        print("\nMissingness (top columns with missing values):")
        missing_summary = []
        for col in list(df_head.columns):
            miss = missing_counts[col]
            non = nonnull_counts[col]
            pct = (miss / (miss+non)) * 100 if (miss+non)>0 else 0.0
            missing_summary.append((pct, col, miss, non))
        missing_summary.sort(reverse=True)
        for pct,col,miss,non in missing_summary[:30]:
            print(f"  {col}: missing={miss}, present={non}, missing%={pct:.2f}%")

        # numeric stats
        if numeric_stats:
            print("\nNumeric column summary (approx):")
            for col,s in numeric_stats.items():
                cnt = s["count"]
                mean = s["sum"]/cnt if cnt>0 else np.nan
                var = (s["sum_sq"]/cnt - mean*mean) if cnt>0 else np.nan
                std = np.sqrt(var) if var>=0 else np.nan
                print(f"  - {col}: count={cnt}, mean={mean:.4f}, std~={std:.4f}, min={s['min']}, max={s['max']}")
        else:
            print("\nNo numeric columns detected in the head sample.")

        # top categorical values (from sampled counts)
        if sample_unique_vals:
            print("\nSampled top values for object/string columns (up to top 10):")
            for col, counter in sample_unique_vals.items():
                if not counter:
                    continue
                top = counter.most_common(MAX_TOP_UNIQUE_TO_SHOW)
                print(f"  - {col}: {top}")
    except Exception as e:
        print(f"Error during chunked processing: {e}")

def summarize_json(path):
    path = Path(path)
    print("="*120)
    print(f"JSON/Dir: {path}")
    if path.is_dir():
        files = list(sorted(path.glob("**/*")))
        json_files = [f for f in files if f.is_file() and f.suffix.lower() in [".json",".geojson",".ndjson"]]
        print(f"Contains {len(json_files)} json-like files (showing up to 5):")
        for f in json_files[:5]:
            print(f" - {f.relative_to(path)} (size={f.stat().st_size} bytes)")
            try:
                with open(f, "r", encoding="utf8") as fh:
                    content = json.load(fh)
                if isinstance(content, dict):
                    keys = list(content.keys())[:20]
                    print(f"   top-level keys: {keys}")
                elif isinstance(content, list) and content:
                    print(f"   list of {len(content)} elements; sample element keys: {list(content[0].keys())[:20]}")
            except Exception as e:
                print(f"   Could not parse JSON sample: {e}")
    else:
        # single json file
        try:
            with open(path, "r", encoding="utf8") as fh:
                content = json.load(fh)
            if isinstance(content, dict):
                print(f"Top-level keys: {list(content.keys())[:50]}")
            elif isinstance(content, list) and content:
                print(f"List with {len(content)} elements; sample keys: {list(content[0].keys())[:50]}")
        except Exception as e:
            print(f"Could not read JSON: {e}")

def main():
    if not ROOT.exists():
        print(f"ERROR: root folder {ROOT} does not exist. Adjust ROOT variable at top of the script.")
        return

    # walk the raw data directory
    for dirpath, dirnames, filenames in os.walk(ROOT):
        rel = Path(dirpath).relative_to(ROOT)
        print("\n" + "-"*100)
        print(f"Directory: {rel}")
        # summarize json directories specially (StatsBomb structure)
        if "events" in dirnames or "matches" in dirnames or any(f.endswith(".json") for f in filenames):
            # if dir contains many JSON files, summarize folder
            if any(f.endswith(".json") or f.endswith(".ndjson") for f in filenames):
                # summarize each json file or dir
                for f in filenames:
                    if f.lower().endswith(".json") or f.lower().endswith(".ndjson"):
                        summarize_json(Path(dirpath)/f)
        # summarize each csv in this directory
        for fn in filenames:
            fp = Path(dirpath)/fn
            if fn.lower().endswith(".csv"):
                summarize_csv(fp)
            elif fn.lower().endswith(".zip"):
                print(f"ZIP: {fp}  (size={fp.stat().st_size} bytes)")
            elif fn.lower().endswith(".json"):
                summarize_json(fp)
            else:
                # small note for other file types
                print(f"Other file: {fp.name} (size={fp.stat().st_size} bytes)")

    print("\n\nSUMMARY: Completed inspection. Copy all printed output and paste it back here for analysis.")
    print("Notes: If any CSV is extremely large and chunking failed, rerun summarize_csv with a smaller CHUNK_SIZE.")

if __name__ == "__main__":
    main()



----------------------------------------------------------------------------------------------------
Directory: .
ZIP: /content/drive/MyDrive/player_value_prediction_project/data/raw/player-scores.zip  (size=172677022 bytes)
ZIP: /content/drive/MyDrive/player_value_prediction_project/data/raw/statsbomb-football-data.zip  (size=1370514785 bytes)
CSV: /content/drive/MyDrive/player_value_prediction_project/data/raw/final_reddit_sentiment.csv
Estimated rows (header excluded): 178
Columns (10): ['player_name', 'season', 'num_posts', 'num_comments_used', 'pos_ratio', 'neu_ratio', 'neg_ratio', 'mean_compound', 'fallback_used', 'subreddits_covered']

Head:
      player_name  season  num_posts  num_comments_used  pos_ratio  neu_ratio  neg_ratio  mean_compound  fallback_used       subreddits_covered
Alberto Gilardino    2004         40                 42        0.0        1.0        0.0       0.288526          False            seriea,soccer
Alberto Gilardino    2005         40                 4

KeyboardInterrupt: 

In [ ]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
from collections import Counter

# Set your raw data root (you said this is the real path)
RAW_ROOT = Path("/content/drive/MyDrive/player_value_prediction_project/data/raw")

CHUNK_SIZE = 200_000
MAX_HEAD = 5

def safe_row_count_csv(path):
    try:
        with open(path, "rb") as f:
            count = 0
            buf = f.raw.read(1024*1024)
            while buf:
                count += buf.count(b"\n")
                buf = f.raw.read(1024*1024)
        return max(0, count - 1)
    except Exception:
        return None

def print_div():
    print("-"*110)


A) player_scores/ folder inspector

In [ ]:
# Paste and run in Colab. Adjust RAW_ROOT if needed.
import pandas as pd
import numpy as np
from pathlib import Path
from collections import Counter

RAW_ROOT = Path("/content/drive/MyDrive/player_value_prediction_project/data/raw/player_scores")
CHUNK_SIZE = 200_000
MAX_HEAD = 5

def inspect_csv(path):
    print("="*100)
    print(f"FILE: {path.name}")
    try:
        est_rows = safe_row_count_csv(path)
        print(f"Estimated rows (header excluded): {est_rows}")
    except:
        print("Row estimate not available.")
    try:
        head = pd.read_csv(path, nrows=MAX_HEAD)
        print(f"Columns ({len(head.columns)}): {list(head.columns)}")
        print("\nHead:")
        print(head.head(MAX_HEAD).to_string(index=False))
        # dtypes snapshot
        for c,t in head.dtypes.items():
            print(f"  {c}: {t}")
    except Exception as e:
        print(f"Could not read head: {e}")
        return

    # chunked summary for missingness + numeric stats for larger files
    try:
        missing = Counter()
        present = Counter()
        numeric_stats = {}
        # determine candidate categorical columns from head
        candidate_cats = [c for c,dt in head.dtypes.items() if dt == object or str(dt).startswith("string")]
        sample_vals = {c: Counter() for c in candidate_cats}

        total = 0
        for chunk in pd.read_csv(path, chunksize=CHUNK_SIZE, low_memory=False):
            total += len(chunk)
            for col in chunk.columns:
                missing[col] += chunk[col].isna().sum()
                present[col] += chunk[col].notna().sum()
            # numeric stats
            for col in chunk.select_dtypes(include=[np.number]).columns:
                arr = chunk[col].dropna().values
                if arr.size==0: continue
                s = numeric_stats.setdefault(col, {"count":0,"sum":0.0,"sum_sq":0.0,"min":float("inf"),"max":float("-inf")})
                s["count"] += arr.size
                s["sum"] += float(arr.sum())
                s["sum_sq"] += float((arr**2).sum())
                s["min"] = min(s["min"], float(arr.min()))
                s["max"] = max(s["max"], float(arr.max()))
            # sample categorical
            for col in candidate_cats:
                vals = chunk[col].dropna().astype(str)
                sample_vals[col].update(vals.value_counts().to_dict())
            # for quick tests you may break early by uncommenting:
            # if total > 500_000: break

        print(f"\nProcessed rows by chunking: {total}")
        print("\nTop missing columns:")
        misslist = []
        for col in head.columns:
            miss = missing[col]
            pres = present[col]
            pct = (miss/(miss+pres))*100 if (miss+pres)>0 else 0
            misslist.append((pct,col,miss,pres))
        misslist.sort(reverse=True)
        for pct,col,miss,pres in misslist[:20]:
            print(f"  {col}: missing={miss} present={pres} ({pct:.2f}%)")

        if numeric_stats:
            print("\nNumeric summaries (approx):")
            for col,s in numeric_stats.items():
                cnt = s["count"]
                mean = s["sum"]/cnt if cnt>0 else np.nan
                var = (s["sum_sq"]/cnt - mean*mean) if cnt>0 else np.nan
                std = np.sqrt(var) if var>=0 else np.nan
                print(f"  {col}: count={cnt}, mean~={mean:.4f}, std~={std:.4f}, min={s['min']}, max={s['max']}")

        print("\nSample top categorical values (up to 10 each):")
        for col,ctr in sample_vals.items():
            if not ctr: continue
            print(f"  {col}: {ctr.most_common(10)}")
    except Exception as e:
        print(f"Chunked summary failed: {e}")

# iterate CSVs
for p in sorted(RAW_ROOT.glob("*.csv")):
    inspect_csv(p)


FILE: appearances.csv
Estimated rows (header excluded): 1706806
Columns (13): ['appearance_id', 'game_id', 'player_id', 'player_club_id', 'player_current_club_id', 'date', 'player_name', 'competition_id', 'yellow_cards', 'red_cards', 'goals', 'assists', 'minutes_played']

Head:
 appearance_id  game_id  player_id  player_club_id  player_current_club_id       date      player_name competition_id  yellow_cards  red_cards  goals  assists  minutes_played
 2231978_38004  2231978      38004             853                     235 2012-07-03 Aurélien Joachim            CLQ             0          0      2        0              90
 2233748_79232  2233748      79232            8841                    2698 2012-07-05   Ruslan Abyshov            ELQ             0          0      0        0              90
 2234413_42792  2234413      42792            6251                     465 2012-07-05      Sander Puri            ELQ             0          0      0        0              45
 2234418_73333  22344

B) injuries/ folder inspector

In [ ]:
# Paste and run in Colab
from pathlib import Path
import pandas as pd
import numpy as np
from collections import Counter

RAW_ROOT = Path("/content/drive/MyDrive/player_value_prediction_project/data/raw/injuries")
CHUNK_SIZE = 200_000
MAX_HEAD = 5

def inspect_csv(path):
    print("="*100)
    print(f"FILE: {path.name}")
    try:
        head = pd.read_csv(path, nrows=MAX_HEAD)
        print(f"Columns ({len(head.columns)}): {list(head.columns)}")
        print("\nHead:")
        print(head.head(MAX_HEAD).to_string(index=False))
    except Exception as e:
        print(f"Could not read head: {e}")
        return

    # compute missingness & sample values (smaller chunking)
    try:
        missing = Counter(); present = Counter(); sample_vals = {c:Counter() for c in head.columns}
        total=0
        for chunk in pd.read_csv(path, chunksize=CHUNK_SIZE, low_memory=False):
            total += len(chunk)
            for col in chunk.columns:
                missing[col] += chunk[col].isna().sum()
                present[col] += chunk[col].notna().sum()
                # sample a few unique values
                sample_vals[col].update(chunk[col].dropna().astype(str).value_counts().to_dict())
            # optional early stop:
            # if total>300_000: break
        print(f"\nProcessed rows by chunking: {total}")
        print("\nMissingness:")
        for col in head.columns:
            miss=missing[col]; pres=present[col]; pct=(miss/(miss+pres))*100 if (miss+pres)>0 else 0
            print(f"  {col}: missing={miss}, present={pres}, missing%={pct:.2f}%")
        print("\nSample top values per column (up to 10):")
        for col,ctr in sample_vals.items():
            print(f"  {col}: {ctr.most_common(10)}")
    except Exception as e:
        print(f"Chunked processing failed: {e}")

# run for csvs in injuries folder
for p in sorted(RAW_ROOT.glob("*.csv")):
    inspect_csv(p)


FILE: player_injuries_impact.csv
Columns (42): ['Name', 'Team Name', 'Position', 'Age', 'Season', 'FIFA rating', 'Injury', 'Date of Injury', 'Date of return', 'Match1_before_injury_Result', 'Match1_before_injury_Opposition', 'Match1_before_injury_GD', 'Match1_before_injury_Player_rating', 'Match2_before_injury_Result', 'Match2_before_injury_Opposition', 'Match2_before_injury_GD', 'Match2_before_injury_Player_rating', 'Match3_before_injury_Result', 'Match3_before_injury_Opposition', 'Match3_before_injury_GD', 'Match3_before_injury_Player_rating', 'Match1_missed_match_Result', 'Match1_missed_match_Opposition', 'Match1_missed_match_GD', 'Match2_missed_match_Result', 'Match2_missed_match_Opposition', 'Match2_missed_match_GD', 'Match3_missed_match_Result', 'Match3_missed_match_Opposition', 'Match3_missed_match_GD', 'Match1_after_injury_Result', 'Match1_after_injury_Opposition', 'Match1_after_injury_GD', 'Match1_after_injury_Player_rating', 'Match2_after_injury_Result', 'Match2_after_injury_

C) statsbomb/ folder inspector

In [ ]:
# Paste and run in Colab
import json
from pathlib import Path

ROOT = Path("/content/drive/MyDrive/player_value_prediction_project/data/raw/statsbomb/data")  # adjust if necessary

print("="*120)
print(f"Listing JSON files under: {ROOT}")
if not ROOT.exists():
    print("Path not found. Adjust the path to your statsbomb json folder.")
else:
    # high-level JSONs
    for f in ["competitions.json"]:
        fp = ROOT/f
        if fp.exists():
            print("\n" + "-"*100)
            print(f"File: {fp.name} size={fp.stat().st_size} bytes")
            try:
                j = json.load(open(fp, "r", encoding="utf8"))
                if isinstance(j, list):
                    print(f"  List of length: {len(j)}. Sample keys from first element:")
                    print(list(j[0].keys())[:50])
                elif isinstance(j, dict):
                    print("  Top-level keys:", list(j.keys())[:50])
            except Exception as e:
                print("  Could not parse:", e)
        else:
            print(f"  {f} not found")

    # inspect directories like events, matches, lineups; show count and sample file names
    for sub in ["events","matches","lineups","three-sixty"]:
        sp = ROOT/sub
        if sp.exists() and sp.is_dir():
            files = sorted([p for p in sp.glob("**/*.json")])
            print("\n" + "-"*80)
            print(f"Directory: {sub}  JSON files: {len(files)}  (showing up to 10 file names)")
            for f in files[:10]:
                print(f"  - {f.name}  size={f.stat().st_size} bytes")
            # attempt to pretty-print keys of first file
            if files:
                try:
                    sample = json.load(open(files[0],"r",encoding="utf8"))
                    if isinstance(sample, list) and sample:
                        print("  Sample element keys:", list(sample[0].keys())[:40])
                    elif isinstance(sample, dict):
                        print("  Sample top-level keys:", list(sample.keys())[:40])
                except Exception as e:
                    print("  Could not parse sample file:", e)
        else:
            print(f"\nDirectory {sub} not present or empty.")


Listing JSON files under: /content/drive/MyDrive/player_value_prediction_project/data/raw/statsbomb/data

----------------------------------------------------------------------------------------------------
File: competitions.json size=32730 bytes
  List of length: 75. Sample keys from first element:
['competition_id', 'season_id', 'country_name', 'competition_name', 'competition_gender', 'competition_youth', 'competition_international', 'season_name', 'match_updated', 'match_updated_360', 'match_available_360', 'match_available']

--------------------------------------------------------------------------------
Directory: events  JSON files: 3464  (showing up to 10 file names)
  - 15946.json  size=3256730 bytes
  - 15956.json  size=2864428 bytes
  - 15973.json  size=3013141 bytes
  - 15978.json  size=3342666 bytes
  - 15986.json  size=3380879 bytes
  - 15998.json  size=3341389 bytes
  - 16010.json  size=3106048 bytes
  - 16023.json  size=3845714 bytes
  - 16029.json  size=3100630 bytes

D) Top / All players + reddit sentiment + other single CSVs

In [ ]:
# Paste and run in Colab
from pathlib import Path
import pandas as pd
import numpy as np

ROOT = Path("/content/drive/MyDrive/player_value_prediction_project/data/raw")
FILES = ["All_players.csv","Top_players.csv","final_reddit_sentiment.csv","player-scores.zip","statsbomb-football-data.zip"]

for fname in FILES:
    fp = ROOT/fname
    if fp.exists():
        print("="*100)
        print(f"File: {fp.name}  size={fp.stat().st_size} bytes")
        if fp.suffix.lower()==".csv":
            try:
                df = pd.read_csv(fp, nrows=5)
                print(f"Columns ({len(df.columns)}): {list(df.columns)}")
                print("Head (up to 5 rows):")
                print(df.head(5).to_string(index=False))
                # quick row estimate
                try:
                    import math
                    rows = safe_row_count_csv(fp)
                    print(f"Estimated rows: {rows}")
                except:
                    pass
            except Exception as e:
                print("Could not read CSV head:", e)
        else:
            print("Not a CSV — zip or other binary file.")
    else:
        print(f"{fname} not present in root.")


File: All_players.csv  size=4754063 bytes
Columns (2): ['player_name', 'season']
Head (up to 5 rows):
    player_name  season
    A.J. Soares    2010
    A.J. Soares    2011
    A.J. Soares    2015
    A.J. Soares    2016
AJ Leitch-Smith    2009
Estimated rows: 239190
File: Top_players.csv  size=12308 bytes
Columns (2): ['player_name', 'season']
Head (up to 5 rows):
      player_name  season
    Achraf Hakimi    2025
  Ademola Lookman    2025
Alberto Gilardino    2004
Alberto Gilardino    2005
Alberto Gilardino    2006
Estimated rows: 650
File: final_reddit_sentiment.csv  size=17391 bytes
Columns (10): ['player_name', 'season', 'num_posts', 'num_comments_used', 'pos_ratio', 'neu_ratio', 'neg_ratio', 'mean_compound', 'fallback_used', 'subreddits_covered']
Head (up to 5 rows):
      player_name  season  num_posts  num_comments_used  pos_ratio  neu_ratio  neg_ratio  mean_compound  fallback_used       subreddits_covered
Alberto Gilardino    2004         40                 42        0.0    

A) MARKET VALUE PREPROCESSING (player_valuations -> player_season targets)

Purpose:
- Load raw `player_valuations.csv` (transfermarkt snapshots),
- Parse dates and map each valuation to a season (season start = July 1),
- Compute multiple candidate player-season target statistics:
    * first_value (first snapshot in season)
    * last_value (last snapshot in season)
    * mean_value, median_value, max_value (peak)
    * count of snapshots in season
    * value_change_pct (last vs first)
    * value_std (volatility)
    * season_start_snapshot (closest to July 1)
    * season_end_snapshot (closest to Jun 30 next year)
    * log1p versions of selected numeric features (log transform)
- Save result to `data/processed/player_valuations_agg.csv`
- Prints a compact summary to help you verify results.

Inputs (expected path in repo):
- /content/drive/MyDrive/player_value_prediction_project/data/raw/player_scores/player_valuations.csv

Outputs:
- /content/drive/MyDrive/player_value_prediction_project/data/processed/player_valuations_agg.csv

Notes:
- Season definition: dates from July 1 (year N) to June 30 (year N+1) map to season = N.
- For "closest to season start/end", tie-breaking picks the earliest in sort order.
- This cell is designed to be idempotent — you can re-run it.

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

# CONFIG - update only if your folder differs
RAW = Path("/content/drive/MyDrive/player_value_prediction_project/data/raw")
PROCESSED = Path("/content/drive/MyDrive/player_value_prediction_project/data/processed")
PROCESSED.mkdir(parents=True, exist_ok=True)

# candidate raw file locations (we look in both)
possible_paths = [
    RAW / "player_scores" / "player_valuations.csv",
    RAW / "player_valuations.csv"
]
player_vals_fp = next((p for p in possible_paths if p.exists()), None)
if player_vals_fp is None:
    raise FileNotFoundError(f"player_valuations.csv not found in any of: {possible_paths}")

print("Loading valuations from:", player_vals_fp)
# Load
df = pd.read_csv(player_vals_fp, parse_dates=["date"], low_memory=False)

# Ensure column names are normalized
df.columns = [c.strip() for c in df.columns]

# Ensure numeric market value
if 'market_value_in_eur' not in df.columns:
    raise KeyError("Expected column 'market_value_in_eur' in player_valuations.csv")
df['market_value_in_eur'] = pd.to_numeric(df['market_value_in_eur'], errors='coerce').fillna(0.0).astype(float)

# Parse dates robustly
df['date'] = pd.to_datetime(df['date'], errors='coerce')
if df['date'].isna().any():
    print("Warning: some valuation rows had unparseable dates; they will get season=NaN. Count:", df['date'].isna().sum())

# Helper: compute season start year for a date (July1..Jun30 -> season = starting year)
def season_from_date(ts):
    if pd.isna(ts):
        return np.nan
    y = ts.year
    return int(y if ts.month >= 7 else y - 1)

df['season'] = df['date'].apply(season_from_date).astype('Int64')

# Drop rows with missing player_id or missing season (can't map)
if 'player_id' not in df.columns:
    raise KeyError("Expected column 'player_id' in player_valuations.csv")
before = len(df)
df = df.dropna(subset=['player_id', 'season'])
after = len(df)
print(f"Dropped {before-after} valuations rows with missing player_id or season.")

# Cast player_id to int
df['player_id'] = df['player_id'].astype(int)

# Basic checks
print("Valuations rows (after clean):", len(df))
print("Distinct players in valuations:", df['player_id'].nunique())
print("Season range:", df['season'].min(), "→", df['season'].max())

# ---------- Aggregations per player-season ----------
group_cols = ['player_id', 'season']

# 1) basic statistical aggregations
agg_basic = df.groupby(group_cols)['market_value_in_eur'].agg(
    snapshots_count = 'count',
    value_min = 'min',
    value_max = 'max',
    value_mean = 'mean',
    value_median = 'median',
    value_std = 'std'
).reset_index()

# 2) first / last snapshots (by date)
df_sorted = df.sort_values(group_cols + ['date'])
first_snap = df_sorted.groupby(group_cols).first().reset_index().rename(columns={
    'date':'first_date', 'market_value_in_eur':'first_value', 'current_club_id':'first_club_id'
})
last_snap = df_sorted.groupby(group_cols).last().reset_index().rename(columns={
    'date':'last_date', 'market_value_in_eur':'last_value', 'current_club_id':'last_club_id'
})

# 3) peak value (already value_max) - keep alias peak_value
agg_basic = agg_basic.merge(first_snap[['player_id','season','first_date','first_value','first_club_id']], on=group_cols, how='left')
agg_basic = agg_basic.merge(last_snap[['player_id','season','last_date','last_value','last_club_id']], on=group_cols, how='left')

# 4) percent change and volatility
agg_basic['value_change_pct'] = np.where(agg_basic['first_value']>0,
                                         (agg_basic['last_value'] - agg_basic['first_value']) / agg_basic['first_value'],
                                         np.nan)
agg_basic['value_std_filled'] = agg_basic['value_std'].fillna(0.0)

# 5) season-start and season-end closest snapshots
# Create season reference dates and compute closest
df['season_start_date'] = df['season'].apply(lambda s: pd.Timestamp(f"{int(s)}-07-01"))
df['season_end_date'] = df['season'].apply(lambda s: pd.Timestamp(f"{int(s)+1}-06-30"))

# compute closest to season start
df['_abs_diff_start'] = (df['date'] - df['season_start_date']).abs()
df_start_pick = df.sort_values(group_cols + ['_abs_diff_start']).groupby(group_cols).first().reset_index()[group_cols + ['market_value_in_eur']].rename(columns={'market_value_in_eur':'value_near_season_start'})

# compute closest to season end
df['_abs_diff_end'] = (df['date'] - df['season_end_date']).abs()
df_end_pick = df.sort_values(group_cols + ['_abs_diff_end']).groupby(group_cols).first().reset_index()[group_cols + ['market_value_in_eur']].rename(columns={'market_value_in_eur':'value_near_season_end'})

# merge
agg = agg_basic.merge(df_start_pick, on=group_cols, how='left').merge(df_end_pick, on=group_cols, how='left')

# 6) log transforms (use log1p to handle zeros)
for c in ['value_min','value_max','value_mean','value_median','first_value','last_value','value_near_season_start','value_near_season_end','value_std_filled']:
    if c in agg.columns:
        agg[f"{c}_log1p"] = np.log1p(agg[c].fillna(0.0))

# 7) tidy columns ordering
cols_order = ['player_id','season','snapshots_count',
              'first_date','first_value','first_value_log1p',
              'last_date','last_value','last_value_log1p',
              'value_near_season_start','value_near_season_start_log1p',
              'value_near_season_end','value_near_season_end_log1p',
              'value_min','value_max','value_mean','value_median','value_std_filled',
              'value_min_log1p','value_max_log1p','value_mean_log1p','value_median_log1p','value_std_filled_log1p',
              'value_change_pct','first_club_id','last_club_id']
# keep only cols that exist
cols_order = [c for c in cols_order if c in agg.columns]
agg = agg[cols_order]

# Save
out_fp = PROCESSED / "player_valuations_agg.csv"
agg.to_csv(out_fp, index=False)
print("Saved aggregated valuations to:", out_fp)

# Print a compact summary for copying
print("\nSAMPLE (first 6 rows):")
print(agg.head(6).to_string(index=False))
print("\nOverall stats:")
print("Total player-season groups:", len(agg))
print("Snapshot counts (per group) summary:")
print(agg['snapshots_count'].describe().to_string())
print("Value range (last_value):", agg['last_value'].min(), "→", agg['last_value'].max())


Loading valuations from: /content/drive/MyDrive/player_value_prediction_project/data/raw/player_scores/player_valuations.csv
Dropped 0 valuations rows with missing player_id or season.
Valuations rows (after clean): 496606
Distinct players in valuations: 31078
Season range: 1999 → 2024
Saved aggregated valuations to: /content/drive/MyDrive/player_value_prediction_project/data/processed/player_valuations_agg.csv

SAMPLE (first 6 rows):
 player_id  season  snapshots_count first_date  first_value  first_value_log1p  last_date  last_value  last_value_log1p  value_near_season_start  value_near_season_start_log1p  value_near_season_end  value_near_season_end_log1p  value_min  value_max   value_mean  value_median  value_std_filled  value_min_log1p  value_max_log1p  value_mean_log1p  value_median_log1p  value_std_filled_log1p  value_change_pct  first_club_id  last_club_id
        10    2004                3 2004-10-04    7000000.0          15.761421 2005-05-05  12000000.0         16.300417    

B) PLAYER PROFILE CLEANING (players.csv -> players_processed.csv)

Purpose:
- Load raw `players.csv` (Transfermarkt player master),
- Parse & standardize key profile fields:
    * date_of_birth -> datetime
    * compute birth_year & age (age at season will be computed when joining to player-season)
    * position & sub_position normalization (simple lowercase)
    * height_in_cm numeric
    * contract_expiration_date parsed where possible (coerce)
    * market_value_in_eur and highest_market_value_in_eur numeric
- Keep a compact set of stable features for merging into player-season master:
    player_id, name, date_of_birth, birth_year, position, sub_position, foot, height_in_cm,
    current_club_id, current_club_domestic_competition_id, market_value_in_eur (snapshot), highest_market_value_in_eur
- Save to data/processed/players_processed.csv
- Prints a compact summary.

Inputs (expected path):
- /content/drive/MyDrive/player_value_prediction_project/data/raw/player_scores/players.csv
  OR /content/drive/MyDrive/player_value_prediction_project/data/raw/players.csv

Outputs:
- /content/drive/MyDrive/player_value_prediction_project/data/processed/players_processed.csv

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

# CONFIG
RAW = Path("/content/drive/MyDrive/player_value_prediction_project/data/raw")
PROCESSED = Path("/content/drive/MyDrive/player_value_prediction_project/data/processed")
PROCESSED.mkdir(parents=True, exist_ok=True)

# find file
possible_players = [
    RAW / "player_scores" / "players.csv",
    RAW / "players.csv"
]
players_fp = next((p for p in possible_players if p.exists()), None)
if players_fp is None:
    raise FileNotFoundError(f"players.csv not found in any of: {possible_players}")

print("Loading players from:", players_fp)
p = pd.read_csv(players_fp, low_memory=False)

# Normalize columns
p.columns = [c.strip() for c in p.columns]

# Keep only useful columns (if present)
keep = [
    'player_id','name','first_name','last_name','date_of_birth','position','sub_position',
    'foot','height_in_cm','current_club_id','current_club_domestic_competition_id',
    'market_value_in_eur','highest_market_value_in_eur','contract_expiration_date'
]
keep_existing = [c for c in keep if c in p.columns]
df = p[keep_existing].copy()

# Parse dates & numeric conversions
if 'date_of_birth' in df.columns:
    df['date_of_birth'] = pd.to_datetime(df['date_of_birth'], errors='coerce')
    df['birth_year'] = df['date_of_birth'].dt.year
else:
    df['date_of_birth'] = pd.NaT
    df['birth_year'] = pd.NA

# height numeric
if 'height_in_cm' in df.columns:
    df['height_in_cm'] = pd.to_numeric(df['height_in_cm'], errors='coerce')

# contract expiration (may be malformed) - try parse ISO-like strings
if 'contract_expiration_date' in df.columns:
    df['contract_expiration_date'] = pd.to_datetime(df['contract_expiration_date'], errors='coerce')

# market values numeric
for col in ['market_value_in_eur','highest_market_value_in_eur']:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Normalize text fields
for col in ['position','sub_position','foot','name','first_name','last_name','current_club_domestic_competition_id']:
    if col in df.columns:
        df[col] = df[col].astype(str).str.strip().replace({'nan':np.nan})

# Compact and dedupe by player_id
if 'player_id' in df.columns:
    df['player_id'] = df['player_id'].astype(int)
    before = len(df)
    df = df.drop_duplicates(subset=['player_id']).reset_index(drop=True)
    after = len(df)
    if before != after:
        print(f"Deduplicated players: {before} -> {after}")

# Save processed players
out_fp = PROCESSED / "players_processed.csv"
df.to_csv(out_fp, index=False)
print("Saved players processed to:", out_fp)

# Print compact summary
print("\nSAMPLE rows:")
print(df.head(6).to_string(index=False))
print("\nCounts & missingness:")
print(df.isna().sum().to_string())


Loading players from: /content/drive/MyDrive/player_value_prediction_project/data/raw/player_scores/players.csv
Saved players processed to: /content/drive/MyDrive/player_value_prediction_project/data/processed/players_processed.csv

SAMPLE rows:
 player_id               name first_name    last_name date_of_birth   position   sub_position  foot  height_in_cm  current_club_id current_club_domestic_competition_id  market_value_in_eur  highest_market_value_in_eur contract_expiration_date  birth_year
        10     Miroslav Klose   Miroslav        Klose    1978-06-09     Attack Centre-Forward right         184.0              398                                  IT1            1000000.0                   30000000.0                      NaT      1978.0
        26 Roman Weidenfeller      Roman Weidenfeller    1980-08-06 Goalkeeper     Goalkeeper  left         190.0               16                                   L1             750000.0                    8000000.0                      NaT  

C) CHUNKED APPEARANCES AGGREGATION (memory-efficient)

Purpose:
- Stream the large appearances.csv in chunks and compute per-player-per-season aggregates
  without loading the whole file into memory.
- Keeps only necessary columns and computes:
    * appearances_count, total_minutes, mean_minutes_agg (computed post-merge),
    * starts_estimate (minutes >= 60), goals, assists, yellow_cards, red_cards,
    * distinct_competitions_count (unique competition ids seen),
    * first_match_date, last_match_date
  Then computes derived metrics: minutes_per_appearance, goals_per90, assists_per90.
- Saves final aggregated file to:
    /content/drive/MyDrive/player_value_prediction_project/data/processed/player_appearances_agg.csv

Why chunked:
- appearances.csv is large (~1.7M rows). Reading in chunks avoids memory blowups and is faster/stable.

Notes:
- Adjust CHUNK_SIZE if you want larger/smaller chunks. 200k is a reasonable default.
- The cell prints progress every N chunks to give you feedback.

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from collections import defaultdict
import math
import time

# CONFIG
RAW = Path("/content/drive/MyDrive/player_value_prediction_project/data/raw")
PROCESSED = Path("/content/drive/MyDrive/player_value_prediction_project/data/processed")
PROCESSED.mkdir(parents=True, exist_ok=True)

# file locations (try common paths)
possible_paths = [
    RAW / "player_scores" / "appearances.csv",
    RAW / "appearances.csv"
]
appearances_fp = next((p for p in possible_paths if p.exists()), None)
if appearances_fp is None:
    raise FileNotFoundError(f"appearances.csv not found in any of: {possible_paths}")

print("Streaming appearances from:", appearances_fp)

# Columns we need (based on previous inspector)
usecols = [
    'player_id','date','competition_id','minutes_played',
    'goals','assists','yellow_cards','red_cards'
]
# Some files list different names; we will handle gracefully by checking header first
with open(appearances_fp, 'r', encoding='utf-8', errors='ignore') as f:
    header_line = f.readline().strip().split(',')
    header = [h.strip() for h in header_line]

# Map available columns
available_cols = [c for c in usecols if c in header]
if not available_cols:
    raise KeyError(f"None of the required columns {usecols} were found in the CSV header: {header[:20]}...")

print("Columns to read (found):", available_cols)

CHUNK_SIZE = 200_000  # adjust down if you still hit memory issues
print("Using chunk size:", CHUNK_SIZE)

# helper: season_from_date
def season_from_date(ts):
    if pd.isna(ts):
        return np.nan
    y = ts.year
    return int(y if ts.month >= 7 else y - 1)

# aggregation dicts
# We'll store numeric sums & counts and min/max dates and a set of competitions per group.
agg_nums = defaultdict(lambda: {
    'appearances_count': 0,
    'total_minutes': 0,
    'starts_estimate': 0,
    'goals': 0,
    'assists': 0,
    'yellow_cards': 0,
    'red_cards': 0,
    'first_match_date': None,
    'last_match_date': None,
    'competitions': set()
})

# read in chunks
reader = pd.read_csv(
    appearances_fp,
    usecols=available_cols,
    parse_dates=['date'],
    chunksize=CHUNK_SIZE,
    low_memory=True,
    encoding='utf-8'
)
t0 = time.time()
chunk_no = 0
for chunk in reader:
    chunk_no += 1
    # normalize columns
    chunk.columns = [c.strip() for c in chunk.columns]
    # drop rows with missing player_id or date
    chunk = chunk.dropna(subset=['player_id','date'])
    # ensure types
    chunk['player_id'] = chunk['player_id'].astype(int)
    chunk['date'] = pd.to_datetime(chunk['date'], errors='coerce')
    chunk['season'] = chunk['date'].apply(season_from_date).astype('Int64')
    # numeric columns fill
    for col in ['minutes_played','goals','assists','yellow_cards','red_cards']:
        if col in chunk.columns:
            chunk[col] = pd.to_numeric(chunk[col], errors='coerce').fillna(0).astype(int)
        else:
            chunk[col] = 0
    # starter heuristic
    chunk['is_starter_est'] = (chunk['minutes_played'] >= 60).astype(int)
    # iterate rows (vectorized groupby would be nicer but we aggregate incrementally)
    # Group by player_id & season in this chunk and aggregate, then merge into agg_nums
    gb = chunk.groupby(['player_id','season'])
    for (player_id, season), sub in gb:
        if pd.isna(season):
            continue
        key = (int(player_id), int(season))
        rec = agg_nums[key]
        rec['appearances_count'] += int(len(sub))
        rec['total_minutes'] += int(sub['minutes_played'].sum())
        rec['starts_estimate'] += int(sub['is_starter_est'].sum())
        rec['goals'] += int(sub['goals'].sum())
        rec['assists'] += int(sub['assists'].sum())
        rec['yellow_cards'] += int(sub['yellow_cards'].sum())
        rec['red_cards'] += int(sub['red_cards'].sum())
        # first / last date
        min_date = sub['date'].min()
        max_date = sub['date'].max()
        if rec['first_match_date'] is None or (pd.notna(min_date) and min_date < rec['first_match_date']):
            rec['first_match_date'] = min_date
        if rec['last_match_date'] is None or (pd.notna(max_date) and max_date > rec['last_match_date']):
            rec['last_match_date'] = max_date
        # competitions
        if 'competition_id' in sub.columns:
            rec['competitions'].update(x for x in sub['competition_id'].unique() if pd.notna(x))
    # progress print
    if chunk_no % 5 == 0:
        elapsed = time.time() - t0
        print(f"Processed {chunk_no} chunks (~{chunk_no*CHUNK_SIZE} rows) — elapsed {elapsed:.0f}s — unique groups so far: {len(agg_nums)}")

# build DataFrame from agg_nums
rows = []
for (player_id, season), rec in agg_nums.items():
    rows.append({
        'player_id': player_id,
        'season': season,
        'appearances_count': rec['appearances_count'],
        'total_minutes': rec['total_minutes'],
        'mean_minutes': (rec['total_minutes']/rec['appearances_count']) if rec['appearances_count']>0 else 0.0,
        'starts_estimate': rec['starts_estimate'],
        'goals': rec['goals'],
        'assists': rec['assists'],
        'yellow_cards': rec['yellow_cards'],
        'red_cards': rec['red_cards'],
        'distinct_competitions_count': len(rec['competitions']),
        'first_match_date': rec['first_match_date'],
        'last_match_date': rec['last_match_date']
    })

agg_df = pd.DataFrame(rows)
# derived metrics
agg_df['minutes_per_appearance'] = agg_df.apply(lambda r: (r['total_minutes']/r['appearances_count']) if r['appearances_count']>0 else 0.0, axis=1)
agg_df['goals_per90'] = agg_df.apply(lambda r: (r['goals'] * 90.0 / r['total_minutes']) if r['total_minutes']>0 else 0.0, axis=1)
agg_df['assists_per90'] = agg_df.apply(lambda r: (r['assists'] * 90.0 / r['total_minutes']) if r['total_minutes']>0 else 0.0, axis=1)
agg_df['played_any'] = (agg_df['appearances_count'] > 0).astype(int)

# finalize dtypes
agg_df['season'] = agg_df['season'].astype(int)
agg_df = agg_df.sort_values(['player_id','season']).reset_index(drop=True)

# Save
out_fp = PROCESSED / "player_appearances_agg.csv"
agg_df.to_csv(out_fp, index=False)
print("\nSaved player appearances aggregates to:", out_fp)
print("Total player-season groups:", len(agg_df))
print("Sample rows:")
print(agg_df.head(6).to_string(index=False))

Streaming appearances from: /content/drive/MyDrive/player_value_prediction_project/data/raw/player_scores/appearances.csv
Columns to read (found): ['player_id', 'date', 'competition_id', 'minutes_played', 'goals', 'assists', 'yellow_cards', 'red_cards']
Using chunk size: 200000
Processed 5 chunks (~1000000 rows) — elapsed 55s — unique groups so far: 53277

Saved player appearances aggregates to: /content/drive/MyDrive/player_value_prediction_project/data/processed/player_appearances_agg.csv
Total player-season groups: 87223
Sample rows:
 player_id  season  appearances_count  total_minutes  mean_minutes  starts_estimate  goals  assists  yellow_cards  red_cards  distinct_competitions_count first_match_date last_match_date  minutes_per_appearance  goals_per90  assists_per90  played_any
        10    2012                 36           2585     71.805556               28     16        3             8          0                            4       2012-08-23      2013-05-26               71.80

D) MERGE: Build initial player-season master dataset (player_season_master.csv)

Purpose:
- Load processed tables:
    - data/processed/players_processed.csv
    - data/processed/player_valuations_agg.csv
    - data/processed/player_appearances_agg.csv
- Merge into a single player-season table keyed by (player_id, season)
- Compute derived meta features:
    - age_at_season_start (season start = July 1 of season)
    - played_any flag, minutes buckets, etc.
- Select a default target:
    - primary: last_value (and last_value_log1p)
    - fallback alternatives kept: value_near_season_end, value_max/value_mean if available
- Fill sensible defaults for missing feature columns
- Save to: data/processed/player_season_master.csv
- Print a compact sample + some diagnostics

Notes:
- You can change target_selection logic later. I recommend `last_value` or `value_near_season_end` for transfer-window forecasting.

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime

# CONFIG - paths
PROJ = Path("/content/drive/MyDrive/player_value_prediction_project")
PROCESSED = PROJ / "data" / "processed"

players_fp = PROCESSED / "players_processed.csv"
vals_fp = PROCESSED / "player_valuations_agg.csv"
apps_fp = PROCESSED / "player_appearances_agg.csv"

# load
print("Loading processed files...")
players = pd.read_csv(players_fp, parse_dates=['date_of_birth'], low_memory=False) if players_fp.exists() else pd.DataFrame()
vals = pd.read_csv(vals_fp, parse_dates=['first_date','last_date'], low_memory=False) if vals_fp.exists() else pd.DataFrame()
apps = pd.read_csv(apps_fp, parse_dates=['first_match_date','last_match_date'], low_memory=False) if apps_fp.exists() else pd.DataFrame()

print("Shapes -> players:", players.shape, "valuations:", vals.shape, "appearances:", apps.shape)

# Ensure 'player_id' and 'season' are present and correctly typed
for df, name in [(vals,'valuations'), (apps,'appearances')]:
    if 'player_id' not in df.columns or 'season' not in df.columns:
        raise KeyError(f"Expected 'player_id' and 'season' in {name} processed file.")

# Cast types
vals['player_id'] = vals['player_id'].astype(int)
vals['season'] = vals['season'].astype(int)
apps['player_id'] = apps['player_id'].astype(int)
apps['season'] = apps['season'].astype(int)
if 'player_id' in players.columns:
    players['player_id'] = players['player_id'].astype(int)

# Merge: start with the appearances (player-season coverage). Then left-join valuations & profile.
master = apps.copy()
# bring valuations (targets/features)
master = master.merge(vals, on=['player_id','season'], how='left', suffixes=('','_val'))

# bring player profile (static)
if not players.empty:
    keep_profile = ['player_id','name','date_of_birth','birth_year','position','sub_position','foot','height_in_cm','current_club_id','current_club_domestic_competition_id']
    keep_profile = [c for c in keep_profile if c in players.columns]
    master = master.merge(players[keep_profile], on='player_id', how='left')

# Derived: season_start date & age_at_season_start
# season_start = July 1 of season
master['season_start_date'] = pd.to_datetime(master['season'].astype(str) + '-07-01', errors='coerce')
# age: if birth_year exists use (season_start_year - birth_year), else try date_of_birth
def compute_age(row):
    if pd.notna(row.get('birth_year')):
        return int(row['season'] - int(row['birth_year']))
    dob = row.get('date_of_birth')
    if pd.notna(dob):
        try:
            born = pd.to_datetime(dob)
            return int((pd.to_datetime(row['season_start_date']) - born).days // 365)
        except:
            return np.nan
    return np.nan

master['age_at_season_start'] = master.apply(compute_age, axis=1)

# Target selection:
# Primary target: last_value (if present), else value_near_season_end, else value_max/value_mean
def pick_target(row):
    if pd.notna(row.get('last_value')):
        return row['last_value']
    if pd.notna(row.get('value_near_season_end')):
        return row['value_near_season_end']
    # fallback to value_max or value_mean
    for c in ['value_max','value_mean','value_median']:
        if c in row and pd.notna(row[c]):
            return row[c]
    return np.nan

master['target_eur'] = master.apply(pick_target, axis=1)
# Also create log1p target
master['target_log1p'] = np.log1p(master['target_eur'].fillna(0.0))

# A convenience column indicating which target was chosen
def target_label(row):
    if pd.notna(row.get('last_value')): return 'last_value'
    if pd.notna(row.get('value_near_season_end')): return 'season_end_snap'
    for c in ['value_max','value_mean','value_median']:
        if c in row and pd.notna(row[c]): return c
    return 'missing'
master['target_choice'] = master.apply(target_label, axis=1)

# Fill common NA for numeric features where appropriate (don't overwrite target)
numeric_zero_fill = ['appearances_count','total_minutes','goals','assists','yellow_cards','red_cards','starts_estimate','distinct_competitions_count']
for c in numeric_zero_fill:
    if c in master.columns:
        master[c] = pd.to_numeric(master[c], errors='coerce').fillna(0)

# Flag popular / played_any (already in appearances)
if 'played_any' not in master.columns:
    master['played_any'] = (master['appearances_count']>0).astype(int)

# Basic normalization: create minutes_per90 if not present
if 'total_minutes' in master.columns and 'goals' in master.columns:
    master['minutes_per90'] = master.apply(lambda r: (r['total_minutes'] / 90.0) if r['total_minutes']>0 else 0.0, axis=1)
    master['goals_per90_simple'] = master.apply(lambda r: (r['goals'] * 90.0 / r['total_minutes']) if r['total_minutes']>0 else 0.0, axis=1)
else:
    master['minutes_per90'] = 0.0
    master['goals_per90_simple'] = 0.0

# Save master
out_fp = PROCESSED / "player_season_master.csv"
master.to_csv(out_fp, index=False)
print("Saved player_season_master to:", out_fp)

# Print compact summary
print("\nSAMPLE rows (first 8):")
cols_preview = [
    'player_id','season','name','age_at_season_start','played_any','appearances_count','total_minutes',
    'goals','assists','target_eur','target_log1p','target_choice'
]
cols_preview = [c for c in cols_preview if c in master.columns]
print(master[cols_preview].head(8).to_string(index=False))

print("\nMaster stats:")
print("Total player-season rows:", len(master))
print("Rows with target available:", master['target_choice'].isin(['last_value','season_end_snap','value_max','value_mean','value_median']).sum(), "/", len(master))
print("Target distribution (top 5 choices):")
print(master['target_choice'].value_counts().head(10).to_string())
print("\nAges (sample):")
if 'age_at_season_start' in master.columns:
    print(master['age_at_season_start'].describe().to_string())


Loading processed files...
Shapes -> players: (32601, 15) valuations: (242272, 26) appearances: (87223, 17)
Saved player_season_master to: /content/drive/MyDrive/player_value_prediction_project/data/processed/player_season_master.csv

SAMPLE rows (first 8):
 player_id  season               name  age_at_season_start  played_any  appearances_count  total_minutes  goals  assists  target_eur  target_log1p target_choice
        10    2012     Miroslav Klose                 34.0           1                 36           2585     16        3   2000000.0     14.508658    last_value
        10    2013     Miroslav Klose                 35.0           1                 29           2220      8        5   1000000.0     13.815512    last_value
        10    2014     Miroslav Klose                 36.0           1                 40           2289     16        9   1000000.0     13.815512    last_value
        10    2015     Miroslav Klose                 37.0           1                 31         

E) MERGE TRANSFERS INTO PLAYER-SEASON MASTER

Purpose:
- Load processed master: data/processed/player_season_master.csv
- Load raw transfers: data/raw/player_scores/transfers.csv (or data/raw/transfers.csv)
- Parse transfer_date -> transfer_season_start (season start year logic: July->season)
- Build per (player_id, season) transfer aggregates:
    * transfers_count
    * transfers_sum_fee, transfers_mean_fee, transfers_max_fee
    * transfers_miss_fee_count (NaNs), transfers_zero_fee_count
    * last_transfer_date_in_season, last_transfer_fee
    * days_since_last_transfer (relative to season_start_date) - if applicable
    * fee_to_market_ratio (sum_fee / target_eur) and max_fee_to_market_ratio
- Merge aggregated features into master and save:
    data/processed/player_season_master_with_transfers.csv
Notes:
- We treat NaN fees separately (missing/disclosed). Zero is interpreted as zero fee (free) and counted separately.
- If a player has multiple transfers in a season we aggregate them.
- If you later want transfer direction (in/out), we can add 'moved_club' flags and club-level features.

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

PROJ = Path("/content/drive/MyDrive/player_value_prediction_project")
RAW = PROJ / "data" / "raw"
PROCESSED = PROJ / "data" / "processed"

# file locations
master_fp = PROCESSED / "player_season_master.csv"
transfers_possible = [
    RAW / "player_scores" / "transfers.csv",
    RAW / "transfers.csv"
]
transfers_fp = next((p for p in transfers_possible if p.exists()), None)
if not master_fp.exists():
    raise FileNotFoundError(f"Master file not found at {master_fp}. Run previous steps first.")
if transfers_fp is None:
    raise FileNotFoundError(f"transfers.csv not found in any of: {transfers_possible}")

print("Loading master from:", master_fp)
master = pd.read_csv(master_fp, parse_dates=['season_start_date'], low_memory=False)
print("Master shape:", master.shape)

print("Loading transfers from:", transfers_fp)
trans = pd.read_csv(transfers_fp, low_memory=False)

# Normalize column names
trans.columns = [c.strip() for c in trans.columns]

# Parse transfer_date robustly
if 'transfer_date' in trans.columns:
    trans['transfer_date'] = pd.to_datetime(trans['transfer_date'], errors='coerce')
else:
    raise KeyError("transfers.csv must contain 'transfer_date' column.")

# Map transfer_date -> season_start (season = year if month >=7 else year-1)
def season_from_date(ts):
    if pd.isna(ts):
        return np.nan
    y = ts.year
    return int(y if ts.month >= 7 else y - 1)

trans['transfer_season'] = trans['transfer_date'].apply(season_from_date).astype('Int64')

# Ensure numeric player_id and transfer_fee
if 'player_id' not in trans.columns:
    raise KeyError("transfers.csv must contain 'player_id' column.")
trans['player_id'] = trans['player_id'].astype(int)

# Normalize transfer_fee numeric; many missing/NaN or 0
if 'transfer_fee' in trans.columns:
    trans['transfer_fee'] = pd.to_numeric(trans['transfer_fee'], errors='coerce')
else:
    trans['transfer_fee'] = np.nan

# Basic aggregation per player-season
group_cols = ['player_id','transfer_season']
agg = trans.groupby(group_cols).agg(
    transfers_count = ('player_id','count'),
    transfers_sum_fee = ('transfer_fee','sum'),
    transfers_mean_fee = ('transfer_fee','mean'),
    transfers_max_fee = ('transfer_fee','max'),
    transfers_missing_fee = ('transfer_fee', lambda s: int(s.isna().sum())),
    transfers_zero_fee = ('transfer_fee', lambda s: int((s==0).sum()))
).reset_index()

# Convert transfer_season col name to 'season' to align with master
agg = agg.rename(columns={'transfer_season':'season'})

# Fill NaN sums/means with zeros where appropriate (but keep missing counts)
agg['transfers_sum_fee'] = agg['transfers_sum_fee'].fillna(0.0)
agg['transfers_mean_fee'] = agg['transfers_mean_fee'].fillna(0.0)
agg['transfers_max_fee'] = agg['transfers_max_fee'].fillna(0.0)

# Last transfer date & fee in the season (helpful feature)
# compute per player-season the last (most recent) transfer_date and its fee
lasts = trans.sort_values(['player_id','transfer_season','transfer_date']).groupby(group_cols).last().reset_index()
lasts = lasts.rename(columns={'transfer_season':'season', 'transfer_date':'last_transfer_date', 'transfer_fee':'last_transfer_fee', 'to_club_id':'last_to_club_id', 'from_club_id':'last_from_club_id'})
lasts = lasts[['player_id','season','last_transfer_date','last_transfer_fee','last_to_club_id','last_from_club_id']]

# merge lasts into agg
agg = agg.merge(lasts, on=['player_id','season'], how='left')

# days_since_last_transfer: relative to season_start_date in master (if exists)
# prepare a small lookup of season_start_date from master for players/seasons present
season_start_map = master.set_index(['player_id','season'])['season_start_date'].to_dict()
def compute_days_since_last(row):
    key = (int(row['player_id']), int(row['season']))
    last_date = row.get('last_transfer_date')
    if pd.isna(last_date):
        return np.nan
    ss = season_start_map.get(key, None)
    if ss is None or pd.isna(ss):
        return np.nan
    # season_start_date may be string; ensure timestamp
    ss_ts = pd.to_datetime(ss)
    return (pd.to_datetime(row['last_transfer_date']) - ss_ts).days

agg['days_since_last_transfer'] = agg.apply(compute_days_since_last, axis=1)

# Merge aggregated transfer features into master (left join)
master_with_trans = master.merge(agg, on=['player_id','season'], how='left')

# Fill 0 for transfer counts where NaN (no transfers)
for c in ['transfers_count','transfers_sum_fee','transfers_mean_fee','transfers_max_fee','transfers_missing_fee','transfers_zero_fee','last_transfer_fee','days_since_last_transfer']:
    if c in master_with_trans.columns:
        if master_with_trans[c].dtype.kind in 'fci':
            master_with_trans[c] = master_with_trans[c].fillna(0)
        else:
            master_with_trans[c] = master_with_trans[c].fillna(0)

# fee_to_market ratios: safe divide (avoid division by zero)
def safe_divide(a,b):
    try:
        return float(a)/float(b) if (b is not None and b != 0 and not pd.isna(b)) else np.nan
    except:
        return np.nan

master_with_trans['sum_fee_to_target_ratio'] = master_with_trans.apply(lambda r: safe_divide(r.get('transfers_sum_fee',np.nan), r.get('target_eur', np.nan)), axis=1)
master_with_trans['max_fee_to_target_ratio'] = master_with_trans.apply(lambda r: safe_divide(r.get('transfers_max_fee',np.nan), r.get('target_eur', np.nan)), axis=1)
master_with_trans['last_fee_to_target_ratio'] = master_with_trans.apply(lambda r: safe_divide(r.get('last_transfer_fee',np.nan), r.get('target_eur', np.nan)), axis=1)

# Save updated master
out_fp = PROCESSED / "player_season_master_with_transfers.csv"
master_with_trans.to_csv(out_fp, index=False)
print("Saved player_season_master_with_transfers to:", out_fp)

# Diagnostics / summary
print("\nMerged shape:", master_with_trans.shape)
print("Players/seasons with at least one transfer in-season:", (master_with_trans['transfers_count']>0).sum())
print("Total transfer sum (all merged rows):", master_with_trans['transfers_sum_fee'].sum())
print("\nSample rows where transfers_count>0 (top 8):")
sample = master_with_trans.loc[master_with_trans['transfers_count']>0, ['player_id','season','transfers_count','transfers_sum_fee','transfers_mean_fee','transfers_max_fee','last_transfer_date','last_transfer_fee','days_since_last_transfer','sum_fee_to_target_ratio']].head(8)
print(sample.to_string(index=False))


Loading master from: /content/drive/MyDrive/player_value_prediction_project/data/processed/player_season_master.csv
Master shape: (87223, 57)
Loading transfers from: /content/drive/MyDrive/player_value_prediction_project/data/raw/player_scores/transfers.csv
Saved player_season_master_with_transfers to: /content/drive/MyDrive/player_value_prediction_project/data/processed/player_season_master_with_transfers.csv

Merged shape: (87223, 71)
Players/seasons with at least one transfer in-season: 20302
Total transfer sum (all merged rows): 51943652000.0

Sample rows where transfers_count>0 (top 8):
 player_id  season  transfers_count  transfers_sum_fee  transfers_mean_fee  transfers_max_fee last_transfer_date  last_transfer_fee  days_since_last_transfer  sum_fee_to_target_ratio
      3333    2015              1.0                0.0                 0.0                0.0         2015-07-01                0.0                       0.0                      0.0
      3333    2023              1.0

In [ ]:
!pip install rapidfuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 27.9 MB/s eta 0:00:00


F) INJURY MERGE: Aggregate player_injuries_impact.csv -> player-season injury features,
then merge into player_season_master_with_transfers.csv

What it does:
- Loads master with transfers: data/processed/player_season_master_with_transfers.csv
- Loads raw injuries: data/raw/injuries/player_injuries_impact.csv
- Attempts to map injury rows (Name) -> player_id using players_processed.csv:
    - exact normalized name match first,
    - optional fuzzy match via rapidfuzz (if installed)
- Extracts season (e.g., '2019/20' -> 2019) and computes days out from Date of Injury to Date of return
- Aggregates per (player_id, season):
    * num_injuries, num_unique_injury_types
    * avg_days_out, num_present_return
    * mean_rating_before, mean_rating_after, avg_rating_drop
- Merges aggregated injury features into master and saves:
    data/processed/player_season_master_with_injuries.csv

Notes:
- The injuries dataset uses textual seasons like '2019/20'. We convert to integer start year.
- Fuzzy matching increases mapping coverage but may need manual verification.
- Review printed diagnostics after running.

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import re
from datetime import datetime

PROJ = Path("/content/drive/MyDrive/player_value_prediction_project")
RAW = PROJ / "data" / "raw"
RAW_INJ = RAW / "injuries"
PROCESSED = PROJ / "data" / "processed"

master_in_fp = PROCESSED / "player_season_master_with_transfers.csv"
players_fp = PROCESSED / "players_processed.csv"
injuries_fp = RAW_INJ / "player_injuries_impact.csv"
out_fp = PROCESSED / "player_season_master_with_injuries.csv"

# --- Safety checks ---
if not master_in_fp.exists():
    raise FileNotFoundError(f"Master with transfers not found at {master_in_fp}. Run transfers merge step first.")
if not players_fp.exists():
    raise FileNotFoundError(f"players_processed.csv not found at {players_fp}. Run players processing step first.")
if not injuries_fp.exists():
    raise FileNotFoundError(f"Injuries file not found at {injuries_fp}.")

print("Loading files...")
master = pd.read_csv(master_in_fp, low_memory=False, parse_dates=['season_start_date'])
players = pd.read_csv(players_fp, low_memory=False, parse_dates=['date_of_birth'])
inj = pd.read_csv(injuries_fp, low_memory=False)

print("Shapes -> master:", master.shape, "players:", players.shape, "injuries:", inj.shape)

# --- Helper functions ---
def normalize_name(s):
    if pd.isna(s):
        return ""
    s = str(s).strip().lower()
    # remove accents & non-word punctuation
    s = re.sub(r"[^\w\s\-']", ' ', s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

# Build player name -> player_id lookup (players_processed has 'name')
players['name_norm'] = players['name'].apply(normalize_name)
name_to_id = players.set_index('name_norm')['player_id'].to_dict()

# Some players have first/last split — also build from first_name + last_name if available
if 'first_name' in players.columns and 'last_name' in players.columns:
    players['fullname_norm'] = (players['first_name'].fillna('') + " " + players['last_name'].fillna('')).apply(normalize_name)
    extra_map = players.set_index('fullname_norm')['player_id'].to_dict()
    # merge into name_to_id for fallback
    for k,v in extra_map.items():
        if k and k not in name_to_id:
            name_to_id[k] = v

print("Player name lookup size (unique names):", len(name_to_id))

# Optional fuzzy matching setup (rapidfuzz)
USE_FUZZY = False
try:
    from rapidfuzz import process, fuzz
    USE_FUZZY = True
    print("rapidfuzz available — fuzzy matching enabled for unmatched names.")
except Exception:
    print("rapidfuzz not available — fuzzy matching disabled. Install via '!pip install rapidfuzz' to enable.")

# Normalize injuries names & map to player_id
inj['Name_norm'] = inj['Name'].apply(normalize_name)

inj['player_id'] = np.nan
matched = 0
for i, row in inj.iterrows():
    nm = row['Name_norm']
    if nm in name_to_id:
        inj.at[i,'player_id'] = name_to_id[nm]
        matched += 1
    else:
        # try fuzzy if enabled
        if USE_FUZZY and nm:
            # look up best match among keys
            choices = list(name_to_id.keys())
            match, score, idx = process.extractOne(nm, choices, scorer=fuzz.token_sort_ratio)
            # conservative threshold 85
            if score >= 85:
                inj.at[i,'player_id'] = name_to_id[match]
                matched += 1
            else:
                inj.at[i,'player_id'] = np.nan
        else:
            inj.at[i,'player_id'] = np.nan

print(f"Mapped injuries -> player_id via name: {matched} / {len(inj)}")

# --- Season parsing in injuries file ---
# Many seasons like '2019/20' -> convert to 2019
def parse_season(s):
    try:
        if pd.isna(s): return np.nan
        s = str(s).strip()
        m = re.match(r"(\d{4})\s*/\s*(\d{2,4})", s)
        if m:
            return int(m.group(1))
        # maybe a single year '2019'
        m2 = re.match(r"(\d{4})", s)
        if m2:
            return int(m2.group(1))
        return np.nan
    except:
        return np.nan

inj['season'] = inj['Season'].apply(parse_season).astype('Int64')

# Parse dates and compute days out where possible
def parse_date(d):
    if pd.isna(d): return pd.NaT
    s = str(d).strip()
    if s.lower() in ['n.a.','na','n.a','nan','none','']:
        return pd.NaT
    if s.lower() in ['present','present ']:
        return pd.NaT
    try:
        # Try common formats
        return pd.to_datetime(s, errors='coerce')
    except:
        return pd.NaT

inj['date_of_injury_parsed'] = inj['Date of Injury'].apply(parse_date)
inj['date_of_return_parsed'] = inj['Date of return'].apply(parse_date)

# Some returns are 'Present' or missing: mark
inj['return_present_flag'] = inj['Date of return'].astype(str).str.lower().str.contains('present', na=False)

# compute days_out if both dates present
def days_out(row):
    a = row['date_of_injury_parsed']
    b = row['date_of_return_parsed']
    if pd.isna(a) or pd.isna(b):
        return np.nan
    try:
        return (b - a).days
    except:
        return np.nan

inj['days_out'] = inj.apply(days_out, axis=1)

# Ratings before/after: columns like 'Match1_before_injury_Player_rating' etc.
rating_before_cols = [c for c in inj.columns if 'before_injury_Player_rating' in c or 'before_injury_Player' in c]
rating_after_cols = [c for c in inj.columns if 'after_injury_Player_rating' in c or 'after_injury_Player' in c]

# fallback known names from earlier sample
possible_before = ['Match1_before_injury_Player_rating','Match2_before_injury_Player_rating','Match3_before_injury_Player_rating']
possible_after = ['Match1_after_injury_Player_rating','Match2_after_injury_Player_rating','Match3_after_injury_Player_rating']
for c in possible_before:
    if c in inj.columns and c not in rating_before_cols:
        rating_before_cols.append(c)
for c in possible_after:
    if c in inj.columns and c not in rating_after_cols:
        rating_after_cols.append(c)

# convert to numeric where possible (coerce strings like '6(S)' -> keep numeric part)
def extract_numeric_rating(x):
    if pd.isna(x): return np.nan
    s = str(x)
    # find first numeric (including decimals)
    m = re.search(r"(\d+(\.\d+)?)", s)
    if m:
        try:
            return float(m.group(1))
        except:
            return np.nan
    return np.nan

for c in rating_before_cols + rating_after_cols:
    inj[c + '_num'] = inj[c].apply(extract_numeric_rating) if c in inj.columns else np.nan

# compute mean before/after per row
def mean_numeric(cols, row):
    vals = []
    for c in cols:
        cn = c + '_num'
        if cn in row.index and not pd.isna(row[cn]):
            vals.append(row[cn])
    if vals:
        return float(np.mean(vals))
    return np.nan

inj['rating_before_mean'] = inj.apply(lambda r: mean_numeric([c for c in rating_before_cols if c in inj.columns], r), axis=1)
inj['rating_after_mean'] = inj.apply(lambda r: mean_numeric([c for c in rating_after_cols if c in inj.columns], r), axis=1)
inj['rating_drop'] = inj['rating_before_mean'] - inj['rating_after_mean']

# Injury type normalization
inj['injury_type'] = inj['Injury'].astype(str).str.strip().str.lower().replace({'nan':np.nan})

# Filter injuries that have both player mapping and season
inj_mapped = inj[~inj['player_id'].isna() & ~inj['season'].isna()].copy()
inj_mapped['player_id'] = inj_mapped['player_id'].astype(int)
inj_mapped['season'] = inj_mapped['season'].astype(int)

print("Injuries rows with mapped player_id and season:", len(inj_mapped), "out of", len(inj))

# --- Aggregations per player-season ---
group_cols = ['player_id','season']
agg_rows = []
for (pid, season), sub in inj_mapped.groupby(group_cols):
    num_inj = len(sub)
    unique_types = sub['injury_type'].nunique(dropna=True)
    avg_days = sub['days_out'].dropna().mean() if sub['days_out'].dropna().size>0 else np.nan
    num_present = int(sub['return_present_flag'].sum())
    mean_before = sub['rating_before_mean'].dropna().mean() if sub['rating_before_mean'].dropna().size>0 else np.nan
    mean_after = sub['rating_after_mean'].dropna().mean() if sub['rating_after_mean'].dropna().size>0 else np.nan
    mean_drop = np.nan
    if pd.notna(mean_before) and pd.notna(mean_after):
        mean_drop = mean_before - mean_after
    agg_rows.append({
        'player_id': int(pid),
        'season': int(season),
        'num_injuries': int(num_inj),
        'num_unique_injury_types': int(unique_types),
        'avg_days_out': float(avg_days) if not pd.isna(avg_days) else np.nan,
        'num_present_return': int(num_present),
        'mean_rating_before': float(mean_before) if not pd.isna(mean_before) else np.nan,
        'mean_rating_after': float(mean_after) if not pd.isna(mean_after) else np.nan,
        'avg_rating_drop': float(mean_drop) if not pd.isna(mean_drop) else np.nan
    })

inj_agg = pd.DataFrame(agg_rows)

print("Aggregated injury groups (player-season):", len(inj_agg))

# Merge into master
master_with_inj = master.merge(inj_agg, on=['player_id','season'], how='left')

# Fill zeros for num_injuries where NaN -> 0
for c in ['num_injuries','num_unique_injury_types','num_present_return']:
    if c in master_with_inj.columns:
        master_with_inj[c] = master_with_inj[c].fillna(0).astype(int)

# Save
master_with_inj.to_csv(out_fp, index=False)
print("Saved player_season_master_with_injuries to:", out_fp)

# Diagnostics
print("\nMerged shape:", master_with_inj.shape)
print("Player-seasons with >=1 injury:", (master_with_inj['num_injuries']>0).sum(), "/", len(master_with_inj))
print("\nSample injury-augmented rows (top 10 where num_injuries>0):")
sample_cols = ['player_id','season','num_injuries','num_unique_injury_types','avg_days_out','num_present_return','mean_rating_before','mean_rating_after','avg_rating_drop']
sample_cols = [c for c in sample_cols if c in master_with_inj.columns]
print(master_with_inj.loc[master_with_inj['num_injuries']>0, sample_cols].head(10).to_string(index=False))


Loading files...
Shapes -> master: (87223, 71) players: (32601, 15) injuries: (656, 42)
Player name lookup size (unique names): 31892
rapidfuzz available — fuzzy matching enabled for unmatched names.
Mapped injuries -> player_id via name: 655 / 656
Injuries rows with mapped player_id and season: 655 out of 656
Aggregated injury groups (player-season): 414
Saved player_season_master_with_injuries to: /content/drive/MyDrive/player_value_prediction_project/data/processed/player_season_master_with_injuries.csv

Merged shape: (87223, 78)
Player-seasons with >=1 injury: 396 / 87223

Sample injury-augmented rows (top 10 where num_injuries>0):
 player_id  season  num_injuries  num_unique_injury_types  avg_days_out  num_present_return  mean_rating_before  mean_rating_after  avg_rating_drop
      8198    2021             2                        1           5.5                   0            7.000000           7.000000         0.000000
     14086    2023             1                        1   

F) MERGE REDDIT SENTIMENT INTO MASTER

- Loads:
    data/processed/player_season_master_with_injuries.csv
    data/raw/final_reddit_sentiment.csv  (or data/processed/final_reddit_sentiment.csv)
- Normalizes names and matches on (name_norm, season).
- Optionally uses rapidfuzz fuzzy matching for unmatched names (threshold 90).
- Saves:
    data/processed/player_season_master_with_sentiment.csv

Notes:
- final_reddit_sentiment.csv columns expected:
    player_name, season, num_posts, num_comments_used, pos_ratio, neu_ratio,
    neg_ratio, mean_compound, fallback_used, subreddits_covered

In [ ]:
import pandas as pd
import numpy as np
import re
from pathlib import Path

PROJ = Path("/content/drive/MyDrive/player_value_prediction_project")
RAW = PROJ / "data" / "raw"
PROCESSED = PROJ / "data" / "processed"

master_in_fp = PROCESSED / "player_season_master_with_injuries.csv"
reddit_possible = [
    RAW / "final_reddit_sentiment.csv",
    PROCESSED / "final_reddit_sentiment.csv"
]
reddit_fp = next((p for p in reddit_possible if p.exists()), None)
out_fp = PROCESSED / "player_season_master_with_sentiment.csv"

if not master_in_fp.exists():
    raise FileNotFoundError(f"Master file not found at {master_in_fp}. Run previous steps first.")
if reddit_fp is None:
    raise FileNotFoundError(f"final_reddit_sentiment.csv not found in {reddit_possible}.")

print("Loading master from:", master_in_fp)
master = pd.read_csv(master_in_fp, low_memory=False)
print("Loading reddit sentiment from:", reddit_fp)
reddit = pd.read_csv(reddit_fp, low_memory=False)

print("Master shape:", master.shape, "Reddit rows:", len(reddit))

# --- Helpers ---
def normalize_name(s):
    if pd.isna(s): return ""
    s = str(s).strip().lower()
    # remove accents (approx) and unwanted chars
    import unicodedata
    s = unicodedata.normalize("NFKD", s)
    s = "".join(ch for ch in s if not unicodedata.combining(ch))
    s = re.sub(r"[^\w\s\-']", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

# normalize names
master['name_norm'] = master['name'].apply(normalize_name) if 'name' in master.columns else ""
reddit['player_name_norm'] = reddit['player_name'].apply(normalize_name)

# prepare keys and quick exact matching
reddit['season'] = reddit['season'].astype(int)
master['season'] = master['season'].astype(int)

# create a lookup from (name_norm,season) -> master index (prefer first occurrence)
master_key_map = {}
for idx, row in master[['name_norm','season']].drop_duplicates().iterrows():
    k = (row['name_norm'], int(row['season']))
    if k not in master_key_map:
        master_key_map[k] = idx

# Try exact matches:
reddit['matched_master_idx'] = reddit.apply(lambda r: master_key_map.get((r['player_name_norm'], int(r['season'])), np.nan), axis=1)

exact_matches = reddit['matched_master_idx'].notna().sum()
print(f"Exact normalized name-season matches: {exact_matches} / {len(reddit)}")

# For unmatched, optionally use fuzzy matching (rapidfuzz)
unmatched = reddit[reddit['matched_master_idx'].isna()].copy()
USE_FUZZY = False
try:
    from rapidfuzz import process, fuzz
    USE_FUZZY = True
    print("rapidfuzz available — attempting conservative fuzzy matches (threshold=90) for unmatched rows.")
except Exception:
    print("rapidfuzz not available — skipping fuzzy matching. Install with '!pip install rapidfuzz' if you want it.")

if USE_FUZZY and len(unmatched) > 0:
    # build list of master name keys
    master_keys = list(master_key_map.keys())
    master_name_choices = [k[0] + "||" + str(k[1]) for k in master_keys]  # include season in string to match season-aware
    # helper: match name+season to choice
    for i, r in unmatched.iterrows():
        target = r['player_name_norm'] + "||" + str(int(r['season']))
        # fuzzy match among master_name_choices; require high score since names can be ambiguous
        match, score, pos = process.extractOne(target, master_name_choices, scorer=fuzz.token_sort_ratio)
        if score >= 90:
            # recover original key and map to master idx
            name_part, season_part = match.split("||", 1)
            key = (name_part, int(season_part))
            reddit.at[i, 'matched_master_idx'] = master_key_map.get(key, np.nan)

fuzzy_matches = reddit['matched_master_idx'].notna().sum() - exact_matches
print(f"Fuzzy matches added: {max(0,int(fuzzy_matches))}")

# Merge sentiment into master by master index
# Build sentiment columns with reddit_ prefix
sent_cols = ['player_name','season','num_posts','num_comments_used','pos_ratio','neu_ratio','neg_ratio','mean_compound','fallback_used','subreddits_covered']
# ensure columns exist
for c in sent_cols:
    if c not in reddit.columns:
        reddit[c] = np.nan

reddit_matched = reddit[reddit['matched_master_idx'].notna()].copy()
reddit_matched['matched_master_idx'] = reddit_matched['matched_master_idx'].astype(int)

# We'll create a DataFrame indexed by master row index to merge easily
sent_by_master_idx = reddit_matched.set_index('matched_master_idx')[['num_posts','num_comments_used','pos_ratio','neu_ratio','neg_ratio','mean_compound','fallback_used','subreddits_covered']]
# rename columns
sent_by_master_idx = sent_by_master_idx.rename(columns={
    'num_posts':'reddit_num_posts',
    'num_comments_used':'reddit_num_comments_used',
    'pos_ratio':'reddit_pos_ratio',
    'neu_ratio':'reddit_neu_ratio',
    'neg_ratio':'reddit_neg_ratio',
    'mean_compound':'reddit_mean_compound',
    'fallback_used':'reddit_fallback_used',
    'subreddits_covered':'reddit_subreddits_covered'
})

# join by index: create a copy to avoid SettingWithCopy warnings
master_out = master.copy()
# create empty cols in master_out
for c in sent_by_master_idx.columns:
    if c not in master_out.columns:
        master_out[c] = np.nan

# assign sentiment values to corresponding rows
for idx, row in sent_by_master_idx.iterrows():
    master_out.loc[idx, row.index] = row.values

# Diagnostics
matched_rows = reddit_matched.shape[0]
print(f"Total reddit rows matched to master: {matched_rows} / {len(reddit)}")
print("Number of master rows populated with reddit data:", master_out['reddit_num_posts'].notna().sum())

# Fill NaNs sensibly for modeling
# numeric ones -> 0, ratios -> 0, fallback_used -> False, subreddits_covered -> ''
numeric_cols = ['reddit_num_posts','reddit_num_comments_used','reddit_pos_ratio','reddit_neu_ratio','reddit_neg_ratio','reddit_mean_compound']
for c in numeric_cols:
    if c in master_out.columns:
        master_out[c] = pd.to_numeric(master_out[c], errors='coerce').fillna(0.0)

if 'reddit_fallback_used' in master_out.columns:
    master_out['reddit_fallback_used'] = master_out['reddit_fallback_used'].fillna(False)

if 'reddit_subreddits_covered' in master_out.columns:
    master_out['reddit_subreddits_covered'] = master_out['reddit_subreddits_covered'].fillna("")

# Save
master_out.to_csv(out_fp, index=False)
print("Saved player_season_master_with_sentiment to:", out_fp)

# Print sample of merged rows with reddit info
print("\nSample merged rows (reddit_num_posts>0):")
if 'reddit_num_posts' in master_out.columns:
    sample = master_out.loc[master_out['reddit_num_posts']>0, ['player_id','season','name','reddit_num_posts','reddit_mean_compound','reddit_pos_ratio','reddit_neu_ratio','reddit_neg_ratio']].head(12)
    if not sample.empty:
        print(sample.to_string(index=False))
    else:
        print("No master rows have reddit_num_posts > 0.")
else:
    print("reddit columns not present in output.")


Loading master from: /content/drive/MyDrive/player_value_prediction_project/data/processed/player_season_master_with_injuries.csv
Loading reddit sentiment from: /content/drive/MyDrive/player_value_prediction_project/data/raw/final_reddit_sentiment.csv
Master shape: (87223, 78) Reddit rows: 178
Exact normalized name-season matches: 73 / 178
rapidfuzz available — attempting conservative fuzzy matches (threshold=90) for unmatched rows.
Fuzzy matches added: 78


/tmp/ipython-input-4096603180.py:125: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'False' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  master_out.loc[idx, row.index] = row.values
/tmp/ipython-input-4096603180.py:125: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'seriea,soccer' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  master_out.loc[idx, row.index] = row.values


Total reddit rows matched to master: 151 / 178
Number of master rows populated with reddit data: 126


/tmp/ipython-input-4096603180.py:140: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  master_out['reddit_fallback_used'] = master_out['reddit_fallback_used'].fillna(False)


Saved player_season_master_with_sentiment to: /content/drive/MyDrive/player_value_prediction_project/data/processed/player_season_master_with_sentiment.csv

Sample merged rows (reddit_num_posts>0):
 player_id  season                  name  reddit_num_posts  reddit_mean_compound  reddit_pos_ratio  reddit_neu_ratio  reddit_neg_ratio
      3109    2012        Steven Gerrard              40.0              0.230425          0.017544          0.982456           0.00000
      3109    2014        Steven Gerrard              40.0              0.230425          0.017544          0.982456           0.00000
      3235    2014         Rio Ferdinand              40.0              0.006857          0.000000          0.976190           0.02381
      3332    2012          Wayne Rooney              40.0              0.265314          0.030303          0.969697           0.00000
      3332    2013          Wayne Rooney              40.0              0.265314          0.030303          0.969697           

G) STATSBOMB PILOT: extract per-player event statistics from first N matches

Place this cell in Colab and run after mounting Google Drive.

Outputs:
 - /content/drive/MyDrive/player_value_prediction_project/data/processed/statsbomb_player_stats_pilot.csv

Config:
 - N_MATCHES: number of matches (events files) to process (default 100)
 - USE_FUZZY_MAPPING: if rapidfuzz is installed, allow conservative fuzzy mapping of names

This pilot extracts (per player-season):
 - shots, xg_sum, goals
 - passes, passes_completed
 - tackles, interceptions, clearances
 - fouls_committed

It attempts to match StatsBomb player names to your Transfermarkt-derived players using normalized names
and optional fuzzy matching (rapidfuzz).

In [ ]:
import json, os, re
from pathlib import Path
from collections import defaultdict
import pandas as pd
import numpy as np
from datetime import datetime

# ---------------- CONFIG ----------------
PROJ = Path("/content/drive/MyDrive/player_value_prediction_project")
STATSBOMB_DATA = PROJ / "data" / "raw" / "statsbomb" / "data"
MATCHES_DIR = STATSBOMB_DATA / "matches"
EVENTS_DIR = STATSBOMB_DATA / "events"
LINEUPS_DIR = STATSBOMB_DATA / "lineups"
PROCESSED = PROJ / "data" / "processed"
PROCESSED.mkdir(parents=True, exist_ok=True)

N_MATCHES = 100   # reduce for quick pilot
USE_FUZZY_MAPPING = True
FUZZY_THRESHOLD = 90

def season_from_date(ts):
    if pd.isna(ts): return np.nan
    y = ts.year
    return int(y if ts.month >= 7 else y - 1)

def normalize_name(s):
    if s is None: return ""
    s = str(s).strip().lower()
    import unicodedata
    s = unicodedata.normalize("NFKD", s)
    s = "".join(ch for ch in s if not unicodedata.combining(ch))
    s = re.sub(r"[^\w\s\-']", " ", s)
    s = re.sub(r"\s+", " ", s)
    return s.strip()

# ---------------- discover files recursively ----------------
def discover_files_recursive(base_dir):
    base = Path(base_dir)
    if not base.exists():
        return {}
    files = list(base.rglob("*.json"))
    mapping = {}
    for p in files:
        # match id is filename without suffix (some filenames may be like '15946.json')
        mid = p.stem
        # prefer first occurrence; if duplicates exist they map to last found (ok for pilot)
        mapping[mid] = p
    return mapping

print("Discovering StatsBomb files (this may take a few seconds)...")
matches_map = discover_files_recursive(MATCHES_DIR)
events_map = discover_files_recursive(EVENTS_DIR)
lineups_map = discover_files_recursive(LINEUPS_DIR)

print(f"Found match files: {len(matches_map)}, event files: {len(events_map)}, lineup files: {len(lineups_map)}")

if len(matches_map)==0 or len(events_map)==0:
    raise FileNotFoundError(f"No match or event JSON files were discovered under {MATCHES_DIR} or {EVENTS_DIR}. Check path and contents.")

# ---------------- load players lookup ----------------
players_fp = PROCESSED / "players_processed.csv"
if not players_fp.exists():
    raise FileNotFoundError(f"players_processed.csv not found at {players_fp}; run players processing earlier.")
players_df = pd.read_csv(players_fp, low_memory=False)
players_df['name_norm'] = players_df['name'].apply(normalize_name)
name_to_pid = players_df.set_index('name_norm')['player_id'].to_dict()

# fuzzy setup
USE_FUZZY = False
if USE_FUZZY_MAPPING:
    try:
        from rapidfuzz import process, fuzz
        USE_FUZZY = True
        print("rapidfuzz available for fuzzy name matching.")
    except Exception:
        print("rapidfuzz not available; fuzzy disabled.")
        USE_FUZZY = False

# ---------------- pilot processing ----------------
processed = 0
matches_processed = 0
skipped_no_event = 0
stats_acc = defaultdict(lambda: defaultdict(float))
map_cache = {}

def extract_xg(ev):
    shot = ev.get('shot') or ev.get('Shot') or {}
    for key in ('statsbomb_xg','shot_statsbomb_xg','shot_xg','xg'):
        if isinstance(shot, dict) and key in shot and shot[key] is not None:
            try:
                return float(shot[key])
            except:
                pass
    return 0.0

def map_statsbomb_player_name(name, team_name, lineup_map=None):
    if name is None:
        return None
    key = (name, team_name)
    if key in map_cache:
        return map_cache[key]
    name_norm = normalize_name(name)
    pid = name_to_pid.get(name_norm)
    if pid is None and lineup_map is not None:
        pid = lineup_map.get(name_norm)
    if pid is None and USE_FUZZY:
        choices = list(name_to_pid.keys())
        if choices:
            match, score, pos = process.extractOne(name_norm, choices, scorer=fuzz.token_sort_ratio)
            if score >= FUZZY_THRESHOLD:
                pid = name_to_pid.get(match)
    map_cache[key] = pid
    return pid

# iterate through matches_map (match ids). Prefer matches that also have event files.
match_ids = [mid for mid in matches_map.keys() if mid in events_map]
if not match_ids:
    # fallback: take intersection by numeric match ids that may be zero-padded differences
    match_ids = list(set(matches_map.keys()) & set(events_map.keys()))

match_ids = sorted(match_ids)[:N_MATCHES]

for match_id in match_ids:
    mpath = matches_map.get(match_id)
    epath = events_map.get(match_id)
    lpath = lineups_map.get(match_id)  # may be None

    if mpath is None or epath is None:
        skipped_no_event += 1
        continue

    try:
        with open(mpath, 'r', encoding='utf-8') as f:
            match_json = json.load(f)
    except Exception as e:
        print("Failed reading match:", match_id, e)
        continue

    # get match_date
    match_date = None
    for dkey in ('match_date','kick_off','utc_date','date'):
        if dkey in match_json:
            try:
                match_date = pd.to_datetime(match_json[dkey])
                break
            except:
                pass
    season = season_from_date(match_date) if match_date is not None else None

    # build lineup map
    lineup_map = {}
    if lpath is not None:
        try:
            with open(lpath, 'r', encoding='utf-8') as f:
                lineup_json = json.load(f)
            # lineup_json could be list
            for team in lineup_json:
                players_list = team.get('lineup', []) or team.get('lineup', [])
                for p in players_list:
                    # p may have 'player' subdict or direct keys
                    pname = None
                    if isinstance(p, dict):
                        if 'player' in p and isinstance(p['player'], dict):
                            pname = p['player'].get('name')
                        else:
                            pname = p.get('name') or p.get('player_name')
                    if pname:
                        pn_norm = normalize_name(pname)
                        pid = name_to_pid.get(pn_norm)
                        if pid:
                            lineup_map[pn_norm] = pid
        except Exception:
            lineup_map = {}

    # load events
    try:
        with open(epath, 'r', encoding='utf-8') as f:
            events = json.load(f)
    except Exception as e:
        print("Failed reading events:", match_id, e)
        continue

    matches_processed += 1
    processed += 1

    for ev in events:
        player_obj = ev.get('player') or {}
        sb_player_name = player_obj.get('name') or ev.get('player_name') or None
        team = ev.get('team',{}) or {}
        sb_team_name = team.get('name') or team.get('team_name') or None
        if sb_player_name is None:
            continue

        pid = map_statsbomb_player_name(sb_player_name, sb_team_name, lineup_map=lineup_map)
        if pid is None:
            continue

        if season is None:
            continue

        key = (int(pid), int(season))
        ev_type = None
        t = ev.get('type') or ev.get('event_type') or {}
        if isinstance(t, dict):
            ev_type = t.get('name')
        else:
            ev_type = t

        # shots/xg/goals
        if ev_type and 'shot' in str(ev_type).lower():
            stats_acc[key]['sb_shots'] += 1
            try:
                stats_acc[key]['sb_xg'] += float(extract_xg(ev))
            except:
                pass
            # goal heuristics
            is_goal = False
            if ev_type and 'goal' in str(ev_type).lower() and 'shot' not in str(ev_type).lower():
                is_goal = True
            shot = ev.get('shot') or {}
            outcome = None
            if isinstance(shot, dict):
                outcome = shot.get('outcome') or {}
                if isinstance(outcome, dict):
                    if outcome.get('name') and 'goal' in outcome.get('name').lower():
                        is_goal = True
                else:
                    if 'goal' in str(outcome).lower():
                        is_goal = True
            if is_goal:
                stats_acc[key]['sb_goals'] += 1

        # passes
        if ev_type and 'pass' in str(ev_type).lower():
            stats_acc[key]['sb_passes'] += 1
            pas = ev.get('pass') or {}
            outcome = pas.get('outcome') if isinstance(pas, dict) else None
            if outcome in (None, {}, ''):
                stats_acc[key]['sb_passes_completed'] += 1
            else:
                if isinstance(outcome, dict):
                    outname = outcome.get('name') or ''
                    if outname.strip() == '':
                        stats_acc[key]['sb_passes_completed'] += 1
                else:
                    if str(outcome).strip() == '':
                        stats_acc[key]['sb_passes_completed'] += 1

        # defensive events
        if ev_type and 'tackle' in str(ev_type).lower():
            stats_acc[key]['sb_tackles'] += 1
        if ev_type and 'interception' in str(ev_type).lower():
            stats_acc[key]['sb_interceptions'] += 1
        if ev_type and 'clearance' in str(ev_type).lower():
            stats_acc[key]['sb_clearances'] += 1
        if ev_type and 'foul' in str(ev_type).lower():
            stats_acc[key]['sb_fouls_committed'] += 1

    if matches_processed % 10 == 0:
        print(f"Processed matches: {matches_processed} / {len(match_ids)} — player-season keys so far: {len(stats_acc)}")

print(f"\nPilot complete — matches processed: {matches_processed}, skipped (no events): {skipped_no_event}")
rows = []
for (pid, season), statmap in stats_acc.items():
    ro


Discovering StatsBomb files (this may take a few seconds)...
Found match files: 48, event files: 3464, lineup files: 3464
rapidfuzz available for fuzzy name matching.

Pilot complete — matches processed: 0, skipped (no events): 0


In [ ]:
"""
STATSBOMB PILOT (events-driven): process first N_EVENTS event JSON files (recursively discovered)
- Uses events/*.json as primary source (these are present for your dataset)
- Uses lineups/*.json when available to help mapping player names -> transfermarkt player_id
- Tries to get match date/season from:
    1) matches/{id}.json if available
    2) first event's 'timestamp' field (common)
    3) fallback: None (skip if no season can be inferred)
- Outputs aggregated per (player_id, season) features to:
    data/processed/statsbomb_player_stats_pilot.csv
"""

import json, os, re
from pathlib import Path
from collections import defaultdict
import pandas as pd
import numpy as np
from datetime import datetime

# -------- CONFIG --------
PROJ = Path("/content/drive/MyDrive/player_value_prediction_project")
STATSBOMB_DATA = PROJ / "data" / "raw" / "statsbomb" / "data"
EVENTS_DIR = STATSBOMB_DATA / "events"
LINEUPS_DIR = STATSBOMB_DATA / "lineups"
MATCHES_DIR = STATSBOMB_DATA / "matches"   # optional
PROCESSED = PROJ / "data" / "processed"
PROCESSED.mkdir(parents=True, exist_ok=True)

N_EVENTS = 100   # number of event files to process (reduce for faster runs)
USE_FUZZY_MAPPING = True
FUZZY_THRESHOLD = 90

def season_from_date(ts):
    if pd.isna(ts): return np.nan
    y = ts.year
    return int(y if ts.month >= 7 else y - 1)

def normalize_name(s):
    if s is None: return ""
    s = str(s).strip().lower()
    import unicodedata
    s = unicodedata.normalize("NFKD", s)
    s = "".join(ch for ch in s if not unicodedata.combining(ch))
    s = re.sub(r"[^\w\s\-']", " ", s)
    s = re.sub(r"\s+", " ", s)
    return s.strip()

# ---------------- discover files recursively ----------------
def discover_files_recursive(base_dir):
    base = Path(base_dir)
    if not base.exists():
        return {}
    files = list(base.rglob("*.json"))
    mapping = {}
    for p in files:
        mid = p.stem
        mapping[mid] = p
    return mapping

print("Discovering StatsBomb event/lineup/match JSONs...")
events_map = discover_files_recursive(EVENTS_DIR)
lineups_map = discover_files_recursive(LINEUPS_DIR)
matches_map = discover_files_recursive(MATCHES_DIR)

print(f"Found: event files={len(events_map)}, lineup files={len(lineups_map)}, match files={len(matches_map)}")

if len(events_map) == 0:
    raise FileNotFoundError(f"No event JSON files found under {EVENTS_DIR} — check your path.")

# ---------------- players lookup ----------------
players_fp = PROCESSED / "players_processed.csv"
if not players_fp.exists():
    raise FileNotFoundError(f"players_processed.csv not found at {players_fp}; run players processing earlier.")
players_df = pd.read_csv(players_fp, low_memory=False)
players_df['name_norm'] = players_df['name'].apply(normalize_name)
name_to_pid = players_df.set_index('name_norm')['player_id'].to_dict()

# optional fuzzy
USE_FUZZY = False
if USE_FUZZY_MAPPING:
    try:
        from rapidfuzz import process, fuzz
        USE_FUZZY = True
        print("rapidfuzz available for fuzzy mapping.")
    except Exception:
        print("rapidfuzz not available; fuzzy disabled.")
        USE_FUZZY = False

# ---------------- pilot processing over event files ----------------
processed = 0
skipped_no_season = 0
stats_acc = defaultdict(lambda: defaultdict(float))
map_cache = {}

def extract_xg(ev):
    shot = ev.get('shot') or ev.get('Shot') or {}
    for key in ('statsbomb_xg','shot_statsbomb_xg','shot_xg','xg'):
        if isinstance(shot, dict) and key in shot and shot[key] is not None:
            try:
                return float(shot[key])
            except:
                pass
    return 0.0

def map_statsbomb_player_name(name, team_name, lineup_map=None):
    if name is None:
        return None
    key = (name, team_name)
    if key in map_cache:
        return map_cache[key]
    name_norm = normalize_name(name)
    pid = name_to_pid.get(name_norm)
    if pid is None and lineup_map is not None:
        pid = lineup_map.get(name_norm)
    if pid is None and USE_FUZZY:
        choices = list(name_to_pid.keys())
        if choices:
            match, score, pos = process.extractOne(name_norm, choices, scorer=fuzz.token_sort_ratio)
            if score >= FUZZY_THRESHOLD:
                pid = name_to_pid.get(match)
    map_cache[key] = pid
    return pid

# choose event ids to process (sorted for reproducibility)
event_ids = sorted(events_map.keys())[:N_EVENTS]

for eid in event_ids:
    epath = events_map.get(eid)
    if epath is None:
        continue

    # build lineup map for this event if available
    lineup_map = {}
    lpath = lineups_map.get(eid)
    if lpath is not None:
        try:
            with open(lpath, 'r', encoding='utf-8') as f:
                lineup_json = json.load(f)
            for team in lineup_json:
                players_list = team.get('lineup', []) or team.get('lineup', [])
                for p in players_list:
                    pname = None
                    if isinstance(p, dict):
                        if 'player' in p and isinstance(p['player'], dict):
                            pname = p['player'].get('name')
                        else:
                            pname = p.get('name') or p.get('player_name')
                    if pname:
                        pn_norm = normalize_name(pname)
                        pid = name_to_pid.get(pn_norm)
                        if pid:
                            lineup_map[pn_norm] = pid
        except Exception:
            lineup_map = {}

    # load events file
    try:
        with open(epath, 'r', encoding='utf-8') as f:
            events = json.load(f)
    except Exception as e:
        print("Error reading events file", epath, e)
        continue

    # infer match_date/season:
    match_date = None
    # try matches_map first
    mpath = matches_map.get(eid)
    if mpath:
        try:
            with open(mpath, 'r', encoding='utf-8') as f:
                mjson = json.load(f)
            for dkey in ('match_date','kick_off','utc_date','date'):
                if dkey in mjson:
                    try:
                        match_date = pd.to_datetime(mjson[dkey])
                        break
                    except:
                        pass
        except Exception:
            match_date = None

    # fallback: use first event timestamp if present
    if match_date is None:
        if isinstance(events, list) and len(events) > 0:
            first_ev = events[0]
            for key in ('timestamp','time','minute'):
                if key in first_ev:
                    ts = first_ev.get('timestamp') or first_ev.get('time')
                    if ts:
                        try:
                            match_date = pd.to_datetime(ts)
                            break
                        except:
                            pass

    season = season_from_date(match_date) if match_date is not None else None
    if season is None:
        skipped_no_season += 1
        # skip this event file for the pilot — we prefer season-aware aggregations
        continue

    processed += 1

    # iterate events
    for ev in events:
        player_obj = ev.get('player') or {}
        sb_player_name = player_obj.get('name') or ev.get('player_name') or None
        team = ev.get('team',{}) or {}
        sb_team_name = team.get('name') or team.get('team_name') or None
        if sb_player_name is None:
            continue

        pid = map_statsbomb_player_name(sb_player_name, sb_team_name, lineup_map=lineup_map)
        if pid is None:
            # skip mapping misses for pilot
            continue

        key = (int(pid), int(season))
        ev_type = None
        t = ev.get('type') or ev.get('event_type') or {}
        if isinstance(t, dict):
            ev_type = t.get('name')
        else:
            ev_type = t

        # shots & xG & goals
        if ev_type and 'shot' in str(ev_type).lower():
            stats_acc[key]['sb_shots'] += 1
            try:
                stats_acc[key]['sb_xg'] += float(extract_xg(ev))
            except:
                pass
            # goal heuristics
            is_goal = False
            if ev_type and 'goal' in str(ev_type).lower() and 'shot' not in str(ev_type).lower():
                is_goal = True
            shot = ev.get('shot') or {}
            if isinstance(shot, dict):
                outcome = shot.get('outcome') or {}
                if isinstance(outcome, dict) and outcome.get('name') and 'goal' in outcome.get('name').lower():
                    is_goal = True
                else:
                    if 'goal' in str(outcome).lower():
                        is_goal = True
            if is_goal:
                stats_acc[key]['sb_goals'] += 1

        # passes
        if ev_type and 'pass' in str(ev_type).lower():
            stats_acc[key]['sb_passes'] += 1
            pas = ev.get('pass') or {}
            outcome = pas.get('outcome') if isinstance(pas, dict) else None
            if outcome in (None, {}, ''):
                stats_acc[key]['sb_passes_completed'] += 1
            else:
                if isinstance(outcome, dict):
                    if (outcome.get('name') or '').strip() == '':
                        stats_acc[key]['sb_passes_completed'] += 1
                else:
                    if str(outcome).strip() == '':
                        stats_acc[key]['sb_passes_completed'] += 1

        # defensive events
        if ev_type and 'tackle' in str(ev_type).lower():
            stats_acc[key]['sb_tackles'] += 1
        if ev_type and 'interception' in str(ev_type).lower():
            stats_acc[key]['sb_interceptions'] += 1
        if ev_type and 'clearance' in str(ev_type).lower():
            stats_acc[key]['sb_clearances'] += 1
        if ev_type and 'foul' in str(ev_type).lower():
            stats_acc[key]['sb_fouls_committed'] += 1

    if processed % 10 == 0:
        print(f"Processed event files: {processed} / {min(N_EVENTS, len(event_ids))} — player-season keys so far: {len(stats_acc)}")

print(f"\nPilot complete. Event files processed: {processed}, skipped(no season inferred): {skipped_no_season}")

# convert stats_acc to DataFrame and save
rows = []
for (pid, season), statmap in stats_acc.items():
    rows.append({
        'player_id': int(pid),
        'season': int(season),
        'sb_shots': int(statmap.get('sb_shots',0)),
        'sb_xg': float(statmap.get('sb_xg',0.0)),
        'sb_goals': int(statmap.get('sb_goals',0)),
        'sb_passes': int(statmap.get('sb_passes',0)),
        'sb_passes_completed': int(statmap.get('sb_passes_completed',0)),
        'sb_tackles': int(statmap.get('sb_tackles',0)),
        'sb_interceptions': int(statmap.get('sb_interceptions',0)),
        'sb_clearances': int(statmap.get('sb_clearances',0)),
        'sb_fouls_committed': int(statmap.get('sb_fouls_committed',0))
    })

sb_df = pd.DataFrame(rows)
out_fp = PROCESSED / "statsbomb_player_stats_pilot.csv"
sb_df.to_csv(out_fp, index=False)
print("Saved pilot stats to:", out_fp)
print("Pilot df shape:", sb_df.shape)
print("Sample (top 12 rows):")
print(sb_df.head(12).to_string(index=False))


Discovering StatsBomb event/lineup/match JSONs...
Found: event files=3464, lineup files=3464, match files=48
rapidfuzz available for fuzzy mapping.
Processed event files: 10 / 100 — player-season keys so far: 25
Processed event files: 20 / 100 — player-season keys so far: 43
Processed event files: 30 / 100 — player-season keys so far: 55
Processed event files: 40 / 100 — player-season keys so far: 118
Processed event files: 50 / 100 — player-season keys so far: 130
Processed event files: 60 / 100 — player-season keys so far: 130
Processed event files: 70 / 100 — player-season keys so far: 130
Processed event files: 80 / 100 — player-season keys so far: 130
Processed event files: 90 / 100 — player-season keys so far: 130
Processed event files: 100 / 100 — player-season keys so far: 130

Pilot complete. Event files processed: 100, skipped(no season inferred): 0
Saved pilot stats to: /content/drive/MyDrive/player_value_prediction_project/data/processed/statsbomb_player_stats_pilot.csv
Pil

H) FULL StatsBomb extraction -> aggregate -> merge pipeline (batch-safe)

Place at top of Colab cell. Adjust CONFIG values as needed.

Outputs:
 - /content/drive/MyDrive/player_value_prediction_project/data/processed/statsbomb_player_stats_batch_{i}.csv
 - /content/drive/MyDrive/player_value_prediction_project/data/processed/statsbomb_player_stats_all.csv
 - /content/drive/MyDrive/player_value_prediction_project/data/processed/statsbomb_player_stats_derived.csv
 - /content/drive/MyDrive/player_value_prediction_project/data/processed/player_season_master_with_statsbomb.csv

Requirements:
 - players_processed.csv (for name -> player_id): data/processed/players_processed.csv
 - player_appearances_agg.csv (for minutes per season): data/processed/player_appearances_agg.csv
 - master with sentiment: data/processed/player_season_master_with_sentiment.csv
 - rapidfuzz (optional): pip install rapidfuzz

In [ ]:
import json, os, re, math, time
from pathlib import Path
from collections import defaultdict
import pandas as pd
import numpy as np
from datetime import datetime
from itertools import islice

# ---------------- CONFIG ----------------
PROJ = Path("/content/drive/MyDrive/player_value_prediction_project")
STATSBOMB_DATA = PROJ / "data" / "raw" / "statsbomb" / "data"
EVENTS_DIR = STATSBOMB_DATA / "events"
LINEUPS_DIR = STATSBOMB_DATA / "lineups"
MATCHES_DIR = STATSBOMB_DATA / "matches"   # optional
PROCESSED = PROJ / "data" / "processed"
PROCESSED.mkdir(parents=True, exist_ok=True)

BATCH_SIZE = 400           # event files per batch (adjust to available runtime)
N_EVENTS_MAX = None        # None -> process all discovered event files; or set an int for quick test
USE_FUZZY_MAPPING = True
FUZZY_THRESHOLD = 90       # conservative

CHECKPOINT_PREFIX = PROCESSED / "statsbomb_player_stats_batch_"
ALL_STATS_FP = PROCESSED / "statsbomb_player_stats_all.csv"
DERIVED_FP = PROCESSED / "statsbomb_player_stats_derived.csv"
MASTER_IN_FP = PROCESSED / "player_season_master_with_sentiment.csv"
MASTER_OUT_FP = PROCESSED / "player_season_master_with_statsbomb.csv"
PLAYERS_FP = PROCESSED / "players_processed.csv"
APPEARANCES_FP = PROCESSED / "player_appearances_agg.csv"

# ---------------- helpers ----------------
def season_from_date(ts):
    if ts is None or (isinstance(ts, float) and math.isnan(ts)): return None
    try:
        y = ts.year
        return int(y if ts.month >= 7 else y - 1)
    except:
        return None

def normalize_name(s):
    if s is None: return ""
    s = str(s).strip().lower()
    import unicodedata
    s = unicodedata.normalize("NFKD", s)
    s = "".join(ch for ch in s if not unicodedata.combining(ch))
    s = re.sub(r"[^\w\s\-']", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def discover_files_recursive(base_dir):
    base = Path(base_dir)
    if not base.exists():
        return {}
    files = list(base.rglob("*.json"))
    mapping = {}
    for p in files:
        mid = p.stem
        mapping[mid] = p
    return mapping

def extract_xg(ev):
    shot = ev.get('shot') or ev.get('Shot') or {}
    for key in ('statsbomb_xg','shot_statsbomb_xg','shot_xg','xg'):
        if isinstance(shot, dict) and key in shot and shot[key] is not None:
            try:
                return float(shot[key])
            except:
                pass
    return 0.0

# ---------------- prepare lookups ----------------
print("Loading players lookup and appearances (for per90) ...")
if not PLAYERS_FP.exists():
    raise FileNotFoundError(f"players_processed.csv not found at {PLAYERS_FP}")
players_df = pd.read_csv(PLAYERS_FP, low_memory=False)
players_df['name_norm'] = players_df['name'].apply(normalize_name)
name_to_pid = players_df.set_index('name_norm')['player_id'].to_dict()

# appearance minutes per (player_id, season)
if not APPEARANCES_FP.exists():
    raise FileNotFoundError(f"player_appearances_agg.csv not found at {APPEARANCES_FP}; run appearances aggregation step.")
ap_df = pd.read_csv(APPEARANCES_FP, low_memory=False)
# keep mapping to quickly lookup minutes
ap_minutes = ap_df.set_index(['player_id','season'])['total_minutes'].to_dict()

# optional fuzzy
USE_FUZZY = False
if USE_FUZZY_MAPPING:
    try:
        from rapidfuzz import process, fuzz
        USE_FUZZY = True
        print("rapidfuzz available for fuzzy mapping.")
    except Exception:
        print("rapidfuzz not available; fuzzy disabled.")
        USE_FUZZY = False

# ---------------- discover StatsBomb files ----------------
print("Discovering StatsBomb files...")
events_map = discover_files_recursive(EVENTS_DIR)
lineups_map = discover_files_recursive(LINEUPS_DIR)
matches_map = discover_files_recursive(MATCHES_DIR)
event_ids = sorted(events_map.keys())
if N_EVENTS_MAX is not None:
    event_ids = event_ids[:N_EVENTS_MAX]

total_events = len(event_ids)
print(f"Found event files: {len(events_map)}, choosing {len(event_ids)} to process.")
print(f"Found lineup files: {len(lineups_map)}, match files: {len(matches_map)}")

# ---------------- batch processing ----------------
map_cache = {}  # (stats_name,team)->player_id cache
stats_acc = defaultdict(lambda: defaultdict(float))  # in-memory aggregator (we'll flush per batch to disk)
batch_no = 0

def map_statsbomb_player_name(name, team_name, lineup_map=None):
    if name is None:
        return None
    key = (name, team_name)
    if key in map_cache:
        return map_cache[key]
    name_norm = normalize_name(name)
    pid = name_to_pid.get(name_norm)
    if pid is None and lineup_map is not None:
        pid = lineup_map.get(name_norm)
    if pid is None and USE_FUZZY:
        choices = list(name_to_pid.keys())
        if choices:
            match, score, pos = process.extractOne(name_norm, choices, scorer=fuzz.token_sort_ratio)
            if score >= FUZZY_THRESHOLD:
                pid = name_to_pid.get(match)
    map_cache[key] = pid
    return pid

def process_event_file(eid):
    epath = events_map.get(eid)
    if epath is None:
        return 0
    # optionally load lineup mapping for this event
    lineup_map = {}
    lpath = lineups_map.get(eid)
    if lpath:
        try:
            with open(lpath, 'r', encoding='utf-8') as f:
                lineup_json = json.load(f)
            for team in lineup_json:
                players_list = team.get('lineup', []) or team.get('lineup', [])
                for p in players_list:
                    pname = None
                    if isinstance(p, dict):
                        if 'player' in p and isinstance(p['player'], dict):
                            pname = p['player'].get('name')
                        else:
                            pname = p.get('name') or p.get('player_name')
                    if pname:
                        pn_norm = normalize_name(pname)
                        pid = name_to_pid.get(pn_norm)
                        if pid:
                            lineup_map[pn_norm] = pid
        except Exception:
            lineup_map = {}

    # try to infer season (prefer matches map)
    match_date = None
    mpath = matches_map.get(eid)
    if mpath:
        try:
            with open(mpath, 'r', encoding='utf-8') as f:
                mjson = json.load(f)
            for dkey in ('match_date','kick_off','utc_date','date'):
                if dkey in mjson:
                    try:
                        match_date = pd.to_datetime(mjson[dkey])
                        break
                    except:
                        pass
        except Exception:
            match_date = None

    # load events
    try:
        with open(epath, 'r', encoding='utf-8') as f:
            events = json.load(f)
    except Exception:
        return 0

    if match_date is None:
        # fallback: try first event timestamp if present
        if isinstance(events, list) and len(events) > 0:
            first_ev = events[0]
            for key in ('timestamp','utc_timestamp','time'):
                ts = first_ev.get(key)
                if ts:
                    try:
                        match_date = pd.to_datetime(ts)
                        break
                    except:
                        pass

    season = season_from_date(match_date) if match_date is not None else None
    if season is None:
        # skip if cannot infer season
        return 0

    # iterate events
    local_count = 0
    for ev in events:
        player_obj = ev.get('player') or {}
        sb_player_name = player_obj.get('name') or ev.get('player_name') or None
        team = ev.get('team',{}) or {}
        sb_team_name = team.get('name') or team.get('team_name') or None
        if sb_player_name is None:
            continue

        pid = map_statsbomb_player_name(sb_player_name, sb_team_name, lineup_map=lineup_map)
        if pid is None:
            continue

        key = (int(pid), int(season))
        ev_type = None
        t = ev.get('type') or ev.get('event_type') or {}
        if isinstance(t, dict):
            ev_type = t.get('name')
        else:
            ev_type = t

        # shots & xG & goals
        if ev_type and 'shot' in str(ev_type).lower():
            stats_acc[key]['sb_shots'] += 1
            try:
                stats_acc[key]['sb_xg'] += float(extract_xg(ev))
            except:
                pass
            # goal heuristics
            is_goal = False
            if ev_type and 'goal' in str(ev_type).lower() and 'shot' not in str(ev_type).lower():
                is_goal = True
            shot = ev.get('shot') or {}
            if isinstance(shot, dict):
                outcome = shot.get('outcome') or {}
                if isinstance(outcome, dict):
                    if outcome.get('name') and 'goal' in outcome.get('name').lower():
                        is_goal = True
                else:
                    if 'goal' in str(outcome).lower():
                        is_goal = True
            if is_goal:
                stats_acc[key]['sb_goals'] += 1

        # passes
        if ev_type and 'pass' in str(ev_type).lower():
            stats_acc[key]['sb_passes'] += 1
            pas = ev.get('pass') or {}
            outcome = pas.get('outcome') if isinstance(pas, dict) else None
            if outcome in (None, {}, ''):
                stats_acc[key]['sb_passes_completed'] += 1
            else:
                if isinstance(outcome, dict):
                    if (outcome.get('name') or '').strip() == '':
                        stats_acc[key]['sb_passes_completed'] += 1
                else:
                    if str(outcome).strip() == '':
                        stats_acc[key]['sb_passes_completed'] += 1

        # defensive events
        if ev_type and 'tackle' in str(ev_type).lower():
            stats_acc[key]['sb_tackles'] += 1
        if ev_type and 'interception' in str(ev_type).lower():
            stats_acc[key]['sb_interceptions'] += 1
        if ev_type and 'clearance' in str(ev_type).lower():
            stats_acc[key]['sb_clearances'] += 1
        if ev_type and 'foul' in str(ev_type).lower():
            stats_acc[key]['sb_fouls_committed'] += 1

        local_count += 1

    return local_count

# iterate in batches
start_time = time.time()
i = 0
event_iter = iter(event_ids)
while True:
    batch_ids = list(islice(event_iter, BATCH_SIZE))
    if not batch_ids:
        break
    batch_no += 1
    batch_start = time.time()
    processed_in_batch = 0
    for eid in batch_ids:
        processed_in_batch += process_event_file(eid)
    # flush current stats_acc to disk as a checkpoint (append or write incremental)
    # Convert stats_acc to DataFrame quickly for saving
    rows = []
    for (pid, season), statmap in stats_acc.items():
        rows.append({
            'player_id': int(pid),
            'season': int(season),
            'sb_shots': int(statmap.get('sb_shots',0)),
            'sb_xg': float(statmap.get('sb_xg',0.0)),
            'sb_goals': int(statmap.get('sb_goals',0)),
            'sb_passes': int(statmap.get('sb_passes',0)),
            'sb_passes_completed': int(statmap.get('sb_passes_completed',0)),
            'sb_tackles': int(statmap.get('sb_tackles',0)),
            'sb_interceptions': int(statmap.get('sb_interceptions',0)),
            'sb_clearances': int(statmap.get('sb_clearances',0)),
            'sb_fouls_committed': int(statmap.get('sb_fouls_committed',0))
        })
    batch_df = pd.DataFrame(rows)
    checkpoint_fp = CHECKPOINT_PREFIX.with_suffix("")  # convert Path to string-like
    checkpoint_fp = Path(str(CHECKPOINT_PREFIX) + f"{batch_no}.csv")
    batch_df.to_csv(checkpoint_fp, index=False)
    batch_elapsed = time.time() - batch_start
    i += len(batch_ids)
    print(f"Batch {batch_no} saved: processed event files so far: {i} / {total_events} (batch time {batch_elapsed:.1f}s). "
          f"Current unique player-season keys: {len(stats_acc)}. Checkpoint: {checkpoint_fp}")

# ---------------- consolidate checkpoints ----------------
print("Consolidating batch checkpoints ...")
# read all batch files in order and take last value per (player_id,season)
batch_files = sorted(Path(PROCESSED).glob("statsbomb_player_stats_batch_*.csv"))
if not batch_files:
    # if no batch files (maybe single small run), use in-memory stats_acc
    rows = []
    for (pid, season), statmap in stats_acc.items():
        rows.append({
            'player_id': int(pid),
            'season': int(season),
            'sb_shots': int(statmap.get('sb_shots',0)),
            'sb_xg': float(statmap.get('sb_xg',0.0)),
            'sb_goals': int(statmap.get('sb_goals',0)),
            'sb_passes': int(statmap.get('sb_passes',0)),
            'sb_passes_completed': int(statmap.get('sb_passes_completed',0)),
            'sb_tackles': int(statmap.get('sb_tackles',0)),
            'sb_interceptions': int(statmap.get('sb_interceptions',0)),
            'sb_clearances': int(statmap.get('sb_clearances',0)),
            'sb_fouls_committed': int(statmap.get('sb_fouls_committed',0))
        })
    all_df = pd.DataFrame(rows)
else:
    # merge incremental: groupby sum across batch files
    parts = []
    for bf in batch_files:
        try:
            p = pd.read_csv(bf, low_memory=False)
            parts.append(p)
        except Exception:
            pass
    if parts:
        all_df = pd.concat(parts, ignore_index=True)
        # groupby sum (some duplicates across batches — sum is correct for counts)
        agg_cols = ['sb_shots','sb_xg','sb_goals','sb_passes','sb_passes_completed','sb_tackles','sb_interceptions','sb_clearances','sb_fouls_committed']
        all_df = all_df.groupby(['player_id','season'], as_index=False)[agg_cols].sum()
    else:
        all_df = pd.DataFrame(columns=['player_id','season','sb_shots','sb_xg','sb_goals','sb_passes','sb_passes_completed','sb_tackles','sb_interceptions','sb_clearances','sb_fouls_committed'])

# save consolidated
all_df.to_csv(ALL_STATS_FP, index=False)
print("Saved consolidated StatsBomb stats to:", ALL_STATS_FP)
print("Consolidated df shape:", all_df.shape)
print("Sample rows:")
print(all_df.head(12).to_string(index=False))

# ---------------- derive rates and per90 features ----------------
print("Computing derived features (pass completion, xg/shots, goals/shots, per90 defensive actions)...")
df = all_df.copy()
# avoid division by zero
df['sb_pass_completion'] = df.apply(lambda r: (r['sb_passes_completed'] / r['sb_passes']) if r['sb_passes']>0 else np.nan, axis=1)
df['sb_xg_per_shot'] = df.apply(lambda r: (r['sb_xg'] / r['sb_shots']) if r['sb_shots']>0 else np.nan, axis=1)
df['sb_goals_per_shot'] = df.apply(lambda r: (r['sb_goals'] / r['sb_shots']) if r['sb_shots']>0 else np.nan, axis=1)

# defensive per90: need minutes from appearances
def per90(stat, pid, season):
    mins = ap_minutes.get((int(pid), int(season)), 0.0)
    if mins > 0:
        return stat * 90.0 / mins
    else:
        return np.nan

df['sb_tackles_per90'] = df.apply(lambda r: per90(r['sb_tackles'], r['player_id'], r['season']), axis=1)
df['sb_interceptions_per90'] = df.apply(lambda r: per90(r['sb_interceptions'], r['player_id'], r['season']), axis=1)
df['sb_clearances_per90'] = df.apply(lambda r: per90(r['sb_clearances'], r['player_id'], r['season']), axis=1)
df['sb_fouls_committed_per90'] = df.apply(lambda r: per90(r['sb_fouls_committed'], r['player_id'], r['season']), axis=1)

# write derived
df.to_csv(DERIVED_FP, index=False)
print("Saved derived StatsBomb features to:", DERIVED_FP)
print("Derived df shape:", df.shape)
print(df.head(12).to_string(index=False))

# ---------------- merge into master ----------------
print("Merging StatsBomb features into master ...")
if not MASTER_IN_FP.exists():
    raise FileNotFoundError(f"Master file {MASTER_IN_FP} not found. Run sentiment merge earlier.")

master = pd.read_csv(MASTER_IN_FP, low_memory=False)
# ensure season dtype match
master['season'] = master['season'].astype(int)
df['season'] = df['season'].astype(int)

# rename sb columns to prefix to avoid collisions (they already have sb_ prefix)
sb_cols = [c for c in df.columns if c not in ['player_id','season']]
# merge (left join master)
master_merged = master.merge(df[['player_id','season'] + sb_cols], how='left', on=['player_id','season'])
# fill NaNs for numeric sb features to 0 (or leave NaN for rates if preferred)
numeric_fill0 = ['sb_shots','sb_xg','sb_goals','sb_passes','sb_passes_completed','sb_tackles','sb_interceptions','sb_clearances','sb_fouls_committed']
for c in numeric_fill0:
    if c in master_merged.columns:
        master_merged[c] = pd.to_numeric(master_merged[c], errors='coerce').fillna(0.0)

# leave per90/rate columns as-is (NaN indicates no stats available)
master_merged.to_csv(MASTER_OUT_FP, index=False)
print("Saved master merged with StatsBomb to:", MASTER_OUT_FP)
print("Final master shape:", master_merged.shape)

# top sample where sb_shots>0
sample_sb = master_merged.loc[master_merged['sb_shots']>0, ['player_id','season','name','sb_shots','sb_xg','sb_goals','sb_passes','sb_pass_completion']].head(20)
if not sample_sb.empty:
    print("\nSample merged rows (players with sb_shots>0):")
    print(sample_sb.to_string(index=False))
else:
    print("\nNo merged rows with sb_shots>0 found in sample (check mapping).")

print("\nAll done. StatsBomb pipeline completed.")


Loading players lookup and appearances (for per90) ...
rapidfuzz available for fuzzy mapping.
Discovering StatsBomb files...
Found event files: 3464, choosing 3464 to process.
Found lineup files: 3464, match files: 48
Batch 1 saved: processed event files so far: 400 / 3464 (batch time 227.3s). Current unique player-season keys: 250. Checkpoint: /content/drive/MyDrive/player_value_prediction_project/data/processed/statsbomb_player_stats_batch_1.csv
Batch 2 saved: processed event files so far: 800 / 3464 (batch time 144.9s). Current unique player-season keys: 691. Checkpoint: /content/drive/MyDrive/player_value_prediction_project/data/processed/statsbomb_player_stats_batch_2.csv
Batch 3 saved: processed event files so far: 1200 / 3464 (batch time 125.7s). Current unique player-season keys: 1234. Checkpoint: /content/drive/MyDrive/player_value_prediction_project/data/processed/statsbomb_player_stats_batch_3.csv
Batch 4 saved: processed event files so far: 1600 / 3464 (batch time 114.6s). 

In [ ]:
import pandas as pd

master_fp = "/content/drive/MyDrive/player_value_prediction_project/data/processed/player_season_master_with_sentiment.csv"
sb_fp = "/content/drive/MyDrive/player_value_prediction_project/data/processed/statsbomb_player_stats_derived.csv"

master = pd.read_csv(master_fp)
sb = pd.read_csv(sb_fp)

print("Master season range:", master['season'].min(), "→", master['season'].max())
print("StatsBomb season range:", sb['season'].min(), "→", sb['season'].max())

# Check overlap
common_keys = pd.merge(
    master[['player_id','season']],
    sb[['player_id','season']],
    how='inner', on=['player_id','season']
)
print("Common (player_id,season) rows:", len(common_keys))

# See which seasons for Miroslav Klose (player_id=10)
print("Master seasons for player_id=10:", master.loc[master['player_id']==10,'season'].unique())
print("StatsBomb seasons for player_id=10:", sb.loc[sb['player_id']==10,'season'].unique())


/tmp/ipython-input-4006128859.py:6: DtypeWarning: Columns (86) have mixed types. Specify dtype option on import or set low_memory=False.
  master = pd.read_csv(master_fp)


Master season range: 2012 → 2024
StatsBomb season range: 2025 → 2025
Common (player_id,season) rows: 0
Master seasons for player_id=10: [2012 2013 2014 2015]
StatsBomb seasons for player_id=10: [2025]


In [ ]:
# DIAGNOSTIC + AUTO-FIX: try shifts on StatsBomb season to maximize overlap, then merge
import pandas as pd
from pathlib import Path

PROJ = Path("/content/drive/MyDrive/player_value_prediction_project")
PROCESSED = PROJ / "data" / "processed"
MASTER_FP = PROCESSED / "player_season_master_with_sentiment.csv"
SB_DERIVED_FP = PROCESSED / "statsbomb_player_stats_derived.csv"
MASTER_OUT_FP = PROCESSED / "player_season_master_with_statsbomb_shifted.csv"

# load
master = pd.read_csv(MASTER_FP, low_memory=False)
sb = pd.read_csv(SB_DERIVED_FP, low_memory=False)

# ensure types
master['season'] = master['season'].astype(int)
sb['season'] = sb['season'].astype(int)

print("Master season range:", master['season'].min(), "→", master['season'].max())
print("StatsBomb season range (before shift):", sb['season'].min(), "→", sb['season'].max())
print("Unique StatsBomb seasons (sample):", sorted(sb['season'].unique())[:10])

# try shifts
shifts = [-1, 0, 1]
overlaps = {}
for s in shifts:
    sb_shift = sb.copy()
    sb_shift['season'] = sb_shift['season'] + s
    merged_keys = pd.merge(master[['player_id','season']], sb_shift[['player_id','season']], how='inner', on=['player_id','season'])
    overlaps[s] = len(merged_keys)
    print(f"Shift {s:+d}: common (player_id,season) rows = {overlaps[s]}")

# pick best shift
best_shift = max(overlaps.items(), key=lambda kv: (kv[1], -abs(kv[0])))[0]
print("\nBest shift selected:", best_shift)

# apply best shift and merge
sb_best = sb.copy()
sb_best['season'] = sb_best['season'] + best_shift

# rename sb columns if needed (they already have sb_ prefix)
sb_cols = [c for c in sb_best.columns if c not in ['player_id','season']]

master_merged = master.merge(sb_best[['player_id','season'] + sb_cols], how='left', on=['player_id','season'])

# fill raw numeric counts with 0 to indicate no events if desired
numeric_fill0 = ['sb_shots','sb_xg','sb_goals','sb_passes','sb_passes_completed','sb_tackles','sb_interceptions','sb_clearances','sb_fouls_committed']
for c in numeric_fill0:
    if c in master_merged.columns:
        master_merged[c] = pd.to_numeric(master_merged[c], errors='coerce').fillna(0.0)

# save
master_merged.to_csv(MASTER_OUT_FP, index=False)
print("Saved adjusted merged master to:", MASTER_OUT_FP)
print("Final master shape:", master_merged.shape)

# report how many rows gained StatsBomb data now
has_shots = master_merged['sb_shots'].fillna(0) > 0
print("Rows in master with sb_shots>0 after shift:", int(has_shots.sum()))
print("Total master rows:", len(master_merged))
print("Proportion with StatsBomb data: {:.4f}".format(has_shots.sum()/len(master_merged)))

# show a few sample merged rows with sb_shots>0
sample = master_merged.loc[has_shots, ['player_id','season','name','sb_shots','sb_xg','sb_goals','sb_passes','sb_pass_completion']].head(30)
if not sample.empty:
    print("\nSample merged rows with sb_shots>0:")
    print(sample.to_string(index=False))
else:
    print("\nNo merged rows with sb_shots>0 found after shifting. We may need to inspect mapping further.")


Master season range: 2012 → 2024
StatsBomb season range (before shift): 2025 → 2025
Unique StatsBomb seasons (sample): [np.int64(2025)]
Shift -1: common (player_id,season) rows = 1086
Shift +0: common (player_id,season) rows = 0
Shift +1: common (player_id,season) rows = 0

Best shift selected: -1
Saved adjusted merged master to: /content/drive/MyDrive/player_value_prediction_project/data/processed/player_season_master_with_statsbomb_shifted.csv
Final master shape: (87223, 103)
Rows in master with sb_shots>0 after shift: 771
Total master rows: 87223
Proportion with StatsBomb data: 0.0088

Sample merged rows with sb_shots>0:
 player_id  season                  name  sb_shots      sb_xg  sb_goals  sb_passes  sb_pass_completion
     12282    2024           Daley Blind      57.0   3.491390      13.0    20961.0            0.819331
     14086    2024          Ashley Young      57.0   6.803828       8.0     6313.0            0.738476
     17259    2024          Manuel Neuer       9.0   7.0515

In [ ]:
"""
Processed-folder tidy/organize helper.

Usage:
- Default: DRY_RUN = True -> only prints what would be moved.
- Set DRY_RUN = False to actually perform the moves.

It will:
- Create: data/processed/master, features, checkpoints, archive
- Move files according to patterns (see report)
- Not delete anything
"""

from pathlib import Path
import shutil
import re
from collections import defaultdict

# ===== CONFIG =====
PROJ = Path("/content/drive/MyDrive/player_value_prediction_project")
PROCESSED = PROJ / "data" / "processed"

DRY_RUN = False   # <-- Set to False to actually move files
VERBOSE = True

# folders to create
DEST_FOLDERS = {
    "master": PROCESSED / "master",
    "features": PROCESSED / "features",
    "checkpoints": PROCESSED / "checkpoints",
    "archive": PROCESSED / "archive"
}

# Files/patterns we *want to keep in master/* (final outputs)
KEEP_IN_MASTER = [
    "player_season_master_with_statsbomb_shifted.csv",
    "player_season_master_with_statsbomb.csv",   # in case existing
    "player_season_master_with_sentiment.csv",
    "player_season_master.csv",
    "players_processed.csv",
    "player_season_master_with_injuries.csv",
    "player_season_master_with_transfers.csv",
    "final_data.csv",
    "final_top_players.csv",
    "final_reddit_sentiment.csv"
]

# Files/patterns for features (useful tables)
FEATURE_PATTERNS = [
    r"player_appearances_agg\.csv",
    r"player_valuations_agg\.csv",
    r"player_appearances_.*\.csv",
    r"statsbomb_player_stats_derived\.csv",
    r"statsbomb_player_stats_all\.csv",
    r"statsbomb_player_stats_pilot\.csv",
    r"player_.*_agg\.csv",
    r"player_valuations_.*\.csv",
    r".*appearances.*\.csv",
    r"players_processed\.csv"
]

# Checkpoints / batch files
CHECKPOINT_PATTERNS = [
    r"statsbomb_player_stats_batch_.*\.csv",
    r"statsbomb_player_stats_pilot_.*\.csv",
    r"statsbomb_player_stats_batch_\d+\.csv",
    r".*_batch_.*\.csv",
    r".*_pilot.*\.csv"
]

# Archive: older master versions or intermediate masters
ARCHIVE_PATTERNS = [
    r".*_with_injuries\.csv",
    r".*_with_transfers\.csv",
    r".*_with_sentiment\.csv",
    r".*_with_.*\.csv",
    r"player_season_master_with_statsbomb_shifted_old.*\.csv",
    r".*debug.*\.csv",
    r".*sample.*\.csv"
]

# Also move large .zip / intermediate files to archive if present
ARCHIVE_EXTS = [".zip", ".gz", ".tar", ".tgz"]

# ===== Create destination folders =====
for k, p in DEST_FOLDERS.items():
    p.mkdir(parents=True, exist_ok=True)

# ===== Scan processed folder =====
all_files = [p for p in PROCESSED.iterdir() if p.is_file()]
to_move = defaultdict(list)
left_in_place = []

# helper to check patterns
def matches_any(name, patterns):
    for pat in patterns:
        if re.fullmatch(pat, name):
            return True
    return False

for f in all_files:
    name = f.name

    # If explicitly keep in master
    if name in KEEP_IN_MASTER:
        left_in_place.append(("master_keep", f))
        continue

    # features
    if matches_any(name, FEATURE_PATTERNS):
        to_move["features"].append(f)
        continue

    # checkpoints
    if matches_any(name, CHECKPOINT_PATTERNS) or "batch" in name.lower() or "checkpoint" in name.lower():
        to_move["checkpoints"].append(f)
        continue

    # archive by pattern
    if matches_any(name, ARCHIVE_PATTERNS) or f.suffix.lower() in ARCHIVE_EXTS:
        to_move["archive"].append(f)
        continue

    # statsbomb/other derived
    if name.startswith("statsbomb_") or name.startswith("sb_") or name.startswith("player_scores"):
        to_move["features"].append(f)
        continue

    # large or legacy masters: move to archive
    if name.startswith("player_season_master") and name not in KEEP_IN_MASTER:
        to_move["archive"].append(f)
        continue

    # otherwise keep in master folder
    to_move["master"].append(f)

# Remove any duplicates across keys (safety)
seen = set()
for k in list(to_move.keys()):
    unique_list = []
    for p in to_move[k]:
        if p.resolve() in seen:
            continue
        unique_list.append(p)
        seen.add(p.resolve())
    to_move[k] = unique_list

# Report planned actions
print("\n=== ORGANIZATION PLAN (DRY_RUN={}) ===\n".format(DRY_RUN))
total_candidates = sum(len(v) for v in to_move.values())
print(f"Total files in {PROCESSED}: {len(all_files)}")
print(f"Planned to move: {total_candidates} files into {len(DEST_FOLDERS)} folders.\n")

for dest, files in to_move.items():
    print(f"-> {dest} ({len(files)} files):")
    if VERBOSE:
        for p in files[:50]:   # limit listing to 50 per group for readability
            print("   ", p.name)
        if len(files) > 50:
            print("   ... (and {} more)".format(len(files)-50))
    print()

# files explicitly left in place (explicit KEEP_IN_MASTER)
if left_in_place:
    print("Files explicitly kept in place (KEEP_IN_MASTER):")
    for tag, p in left_in_place:
        print("   ", p.name)
    print()

# Confirmation & execution
if DRY_RUN:
    print("DRY_RUN=True -> no files moved. Inspect the plan above. Set DRY_RUN=False and re-run to execute moves.")
else:
    moved = []
    for dest_key, files in to_move.items():
        dest_dir = DEST_FOLDERS.get(dest_key, PROCESSED/dest_key)
        dest_dir.mkdir(parents=True, exist_ok=True)
        for src in files:
            tgt = dest_dir / src.name
            try:
                shutil.move(str(src), str(tgt))
                moved.append((src.name, str(tgt)))
            except Exception as e:
                print("ERROR moving", src, "->", tgt, ":", e)

    print("\nMoved files summary ({} moved):".format(len(moved)))
    for sname, t in moved:
        print("  -", sname, "->", t)
    print("\nDone. Review the folders under:", PROCESSED)



=== ORGANIZATION PLAN (DRY_RUN=False) ===

Total files in /content/drive/MyDrive/player_value_prediction_project/data/processed: 21
Planned to move: 14 files into 4 folders.

-> features (5 files):
    player_valuations_agg.csv
    player_appearances_agg.csv
    statsbomb_player_stats_pilot.csv
    statsbomb_player_stats_all.csv
    statsbomb_player_stats_derived.csv

-> checkpoints (9 files):
    statsbomb_player_stats_batch_1.csv
    statsbomb_player_stats_batch_2.csv
    statsbomb_player_stats_batch_3.csv
    statsbomb_player_stats_batch_4.csv
    statsbomb_player_stats_batch_5.csv
    statsbomb_player_stats_batch_6.csv
    statsbomb_player_stats_batch_7.csv
    statsbomb_player_stats_batch_8.csv
    statsbomb_player_stats_batch_9.csv

Files explicitly kept in place (KEEP_IN_MASTER):
    players_processed.csv
    player_season_master.csv
    player_season_master_with_transfers.csv
    player_season_master_with_injuries.csv
    player_season_master_with_sentiment.csv
    player_seas

In [ ]:
# Move canonical master files into processed/master/ (DRY RUN by default)
from pathlib import Path
import shutil

PROJ = Path("/content/drive/MyDrive/player_value_prediction_project")
PROCESSED = PROJ / "data" / "processed"
MASTER_DIR = PROCESSED / "master"
MASTER_DIR.mkdir(parents=True, exist_ok=True)

DRY_RUN = False   # <- set to False to actually move files

master_files = [
    "players_processed.csv",
    "player_season_master.csv",
    "player_season_master_with_transfers.csv",
    "player_season_master_with_injuries.csv",
    "player_season_master_with_sentiment.csv",
    "player_season_master_with_statsbomb.csv",
    "player_season_master_with_statsbomb_shifted.csv"
]

print("Source directory:", PROCESSED)
print("Target master directory:", MASTER_DIR)
print("\n--- BEFORE (files currently in processed/) ---")
for p in sorted(PROCESSED.iterdir()):
    if p.is_file():
        print("  ", p.name)

to_move = []
for fname in master_files:
    src = PROCESSED / fname
    if src.exists():
        dst = MASTER_DIR / fname
        to_move.append((src, dst))
    else:
        print(f"(missing) {fname} not found in processed/ — skipping")

print("\nPlanned moves ({} files):".format(len(to_move)))
for s, d in to_move:
    print("  ", s.name, "->", d)

if DRY_RUN:
    print("\nDRY_RUN=True -> no files moved. Inspect the planned moves above.")
else:
    moved = []
    for s, d in to_move:
        try:
            # if file already exists at destination, add suffix to avoid accidental overwrite
            if d.exists():
                import time
                suffix = f".backup.{int(time.time())}"
                d = d.with_name(d.name + suffix)
            shutil.move(str(s), str(d))
            moved.append((s.name, str(d)))
        except Exception as e:
            print("ERROR moving", s, "->", d, ":", e)
    print("\nMoved files:")
    for sname, dpath in moved:
        print("  -", sname, "->", dpath)

print("\n--- AFTER (files now in processed/master/) ---")
for p in sorted(MASTER_DIR.iterdir()):
    if p.is_file():
        print("  ", p.name)


Source directory: /content/drive/MyDrive/player_value_prediction_project/data/processed
Target master directory: /content/drive/MyDrive/player_value_prediction_project/data/processed/master

--- BEFORE (files currently in processed/) ---
   player_season_master.csv
   player_season_master_with_injuries.csv
   player_season_master_with_sentiment.csv
   player_season_master_with_statsbomb.csv
   player_season_master_with_statsbomb_shifted.csv
   player_season_master_with_transfers.csv
   players_processed.csv

Planned moves (7 files):
   players_processed.csv -> /content/drive/MyDrive/player_value_prediction_project/data/processed/master/players_processed.csv
   player_season_master.csv -> /content/drive/MyDrive/player_value_prediction_project/data/processed/master/player_season_master.csv
   player_season_master_with_transfers.csv -> /content/drive/MyDrive/player_value_prediction_project/data/processed/master/player_season_master_with_transfers.csv
   player_season_master_with_injuries.

In [ ]:
# Zip checkpoints directory (creates archive but DOES NOT delete originals)
from pathlib import Path
import shutil

PROJ = Path("/content/drive/MyDrive/player_value_prediction_project")
PROCESSED = PROJ / "data" / "processed"
CHK = PROCESSED / "checkpoints"
ARCHIVE_DIR = PROCESSED / "archive"
ARCHIVE_DIR.mkdir(parents=True, exist_ok=True)
ARCHIVE_ZIP = ARCHIVE_DIR / "checkpoints_archive.zip"

if not CHK.exists() or not any(CHK.iterdir()):
    print("Checkpoints folder is empty or missing:", CHK)
else:
    print("Creating zip:", ARCHIVE_ZIP)
    shutil.make_archive(str(ARCHIVE_ZIP.with_suffix('')), 'zip', root_dir=str(CHK))
    print("Created zip:", ARCHIVE_ZIP)
    print("You can remove originals manually after verifying the archive contents.")


Creating zip: /content/drive/MyDrive/player_value_prediction_project/data/processed/archive/checkpoints_archive.zip
Created zip: /content/drive/MyDrive/player_value_prediction_project/data/processed/archive/checkpoints_archive.zip
You can remove originals manually after verifying the archive contents.


I) FINAL TRAINING MASTER ASSEMBLY
 - Loads canonical enriched master (with transfers, injuries, reddit, statsbomb)
 - Creates a few derived features (has_statsbomb, per90 rates, age buckets)
 - Fills missing values sensibly for modeling
 - Saves final CSV at: data/processed/master/final_training_master.csv


In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

PROJ = Path("/content/drive/MyDrive/player_value_prediction_project")
MASTER_DIR = PROJ / "data" / "processed" / "master"
MASTER_IN = MASTER_DIR / "player_season_master_with_statsbomb_shifted.csv"
MASTER_OUT = MASTER_DIR / "final_training_master.csv"

print("Loading master:", MASTER_IN)
master = pd.read_csv(MASTER_IN, low_memory=False)

# ensure dtypes
if 'season' in master.columns:
    master['season'] = master['season'].astype(int)
if 'player_id' in master.columns:
    master['player_id'] = master['player_id'].astype(int)

# 1) has_statsbomb flag
if 'sb_shots' in master.columns:
    master['has_statsbomb'] = master['sb_shots'].fillna(0) > 0
else:
    master['has_statsbomb'] = False

# 2) per-appearance minutes if not present
if 'minutes_per_appearance' not in master.columns and {'total_minutes','appearances_count'}.issubset(master.columns):
    master['minutes_per_appearance'] = master.apply(
        lambda r: (r['total_minutes']/r['appearances_count']) if (r['appearances_count']>0 and not pd.isna(r['total_minutes'])) else np.nan,
        axis=1
    )

# 3) goals/assists per90 if missing (use total_minutes)
def safe_per90(numerator_col):
    out_col = f"{numerator_col}_per90"
    if numerator_col in master.columns and out_col not in master.columns:
        master[out_col] = master.apply(
            lambda r: (r[numerator_col]*90.0/r['total_minutes']) if (('total_minutes' in master.columns) and (not pd.isna(r.get('total_minutes')) and r['total_minutes']>0)) else np.nan,
            axis=1
        )

for c in ['goals','assists','yellow_cards','red_cards']:
    safe_per90(c)

# 4) make sure core StatsBomb numeric fields exist and fill zeros where appropriate
sb_numeric = ['sb_shots','sb_xg','sb_goals','sb_passes','sb_passes_completed','sb_tackles','sb_interceptions','sb_clearances','sb_fouls_committed']
for c in sb_numeric:
    if c in master.columns:
        master[c] = pd.to_numeric(master[c], errors='coerce').fillna(0.0)
    else:
        master[c] = 0.0

# 5) Derived ratio features (if present)
if 'sb_passes' in master.columns and 'sb_passes_completed' in master.columns:
    master['sb_pass_completion'] = master.apply(lambda r: (r['sb_passes_completed']/r['sb_passes']) if r['sb_passes']>0 else np.nan, axis=1)

# 6) fill rate columns with a sentinel (so models can learn missingness) or leave as-is
rate_cols = [c for c in master.columns if any(s in c for s in ['_per90','pass_completion','xg_per_shot','goals_per_shot'])]
for c in rate_cols:
    master[c] = master[c].replace([np.inf, -np.inf], np.nan)
    # sentinel for missing/per90 rates
    master[c] = master[c].fillna(-1.0)

# 7) age bucket (if age present)
if 'age_at_season_start' in master.columns:
    master['age_bucket'] = pd.cut(master['age_at_season_start'],
                                  bins=[0,18,21,24,27,30,33,40,100],
                                  labels=['<18','18-20','21-23','24-26','27-29','30-32','33-39','40+'],
                                  include_lowest=True)

# 8) small helper features (presence flags)
master['has_market_value'] = (~master['target_eur'].isna()) if 'target_eur' in master.columns else False
master['played_any_flag'] = master['played_any'].fillna(0).astype(int) if 'played_any' in master.columns else 0

# 9) keep columns order tidy: preference core ids -> static player attrs -> appearances/val -> sb features -> sentiment/injury/transfer -> derived
cols_priority = []
for c in ['player_id','season','name','age_at_season_start','position','sub_position','current_club_id','current_club_domestic_competition_id','played_any','appearances_count','total_minutes','minutes_per_appearance','goals','assists','goals_per90','assists_per90','target_eur','target_log1p','target_choice']:
    if c in master.columns:
        cols_priority.append(c)

sb_cols = [c for c in master.columns if c.startswith('sb_')]
sent_cols = [c for c in master.columns if c.startswith('reddit_') or c.startswith('reddit')]
inj_cols = [c for c in master.columns if 'inj' in c or 'injury' in c]
trans_cols = [c for c in master.columns if 'transfer' in c]

# add remaining columns (keep them but put after priority)
remaining = [c for c in master.columns if c not in cols_priority + sb_cols + sent_cols + inj_cols + trans_cols]
final_cols = cols_priority + sb_cols + sent_cols + inj_cols + trans_cols + remaining

# ensure uniqueness and existing columns
final_cols = [c for i,c in enumerate(final_cols) if c in master.columns and c not in final_cols[:i]]

master = master[final_cols]

# Save
master.to_csv(MASTER_OUT, index=False)
print("Saved final training master to:", MASTER_OUT)
print("Shape:", master.shape)
print("Sample columns (first 60):")
print(master.columns.tolist()[:60])


Loading master: /content/drive/MyDrive/player_value_prediction_project/data/processed/master/player_season_master_with_statsbomb_shifted.csv
Saved final training master to: /content/drive/MyDrive/player_value_prediction_project/data/processed/master/final_training_master.csv
Shape: (87223, 109)
Sample columns (first 60):
['player_id', 'season', 'name', 'age_at_season_start', 'position', 'sub_position', 'current_club_id', 'current_club_domestic_competition_id', 'played_any', 'appearances_count', 'total_minutes', 'minutes_per_appearance', 'goals', 'assists', 'goals_per90', 'assists_per90', 'target_eur', 'target_log1p', 'target_choice', 'sb_shots', 'sb_xg', 'sb_goals', 'sb_passes', 'sb_passes_completed', 'sb_tackles', 'sb_interceptions', 'sb_clearances', 'sb_fouls_committed', 'sb_pass_completion', 'sb_xg_per_shot', 'sb_goals_per_shot', 'sb_tackles_per90', 'sb_interceptions_per90', 'sb_clearances_per90', 'sb_fouls_committed_per90', 'reddit_num_posts', 'reddit_num_comments_used', 'reddit_po

In [ ]:
# SAFE PREVIEW: Inspect master file before any modification
import pandas as pd
import numpy as np
from pathlib import Path
import re

PROJ = Path("/content/drive/MyDrive/player_value_prediction_project")
MASTER_DIR = PROJ / "data" / "processed" / "master"
MASTER_IN = MASTER_DIR / "final_training_master.csv"

print("File path:", MASTER_IN)
df = pd.read_csv(MASTER_IN, low_memory=False)
print("Loaded shape:", df.shape)
print("\nColumns (first 80):\n", df.columns.tolist()[:80])

# Check key ids / target
for col in ['player_id','season','name','target_log1p']:
    print(f" - {col}: Present? {col in df.columns}")

# Show sample season values and attempt to parse years
sample_seasons = df['season'].dropna().astype(str).unique()[:20] if 'season' in df.columns else []
print("\nSample season values (first 20):", sample_seasons)

def extract_season_start_year(s):
    try:
        if pd.isna(s): return np.nan
        s = str(s)
        for part in [s.split('/')[0], s.split('-')[0]]:
            if part.isdigit() and len(part) == 4: return int(part)
        m = re.search(r'20\d{2}', s)
        if m: return int(m.group(0))
    except: return np.nan
    return np.nan

if 'season' in df.columns:
    df_preview = df[['player_id','season']].drop_duplicates().head(20).copy()
    df_preview['season_start_year_guess'] = df_preview['season'].apply(extract_season_start_year)
    print("\nPreview season parsing (player_id, season, guessed_start_year):")
    print(df_preview.head(20).to_string(index=False))

# Check basic counts: unique players, seasons
if 'player_id' in df.columns:
    print("\nUnique players:", df['player_id'].nunique())
if 'season' in df.columns:
    print("Unique seasons:", df['season'].nunique())

# Quick column existence for features we expect to scale or lag
expected_numeric = ['age_at_season_start','total_minutes','goals','assists','appearances_count','reddit_mean_compound','num_injuries','transfers_count','transfers_sum_fee']
print("\nExpected numeric present:")
for c in expected_numeric:
    print(f"  {c}: {c in df.columns}")

# Show a few rows to inspect types/NaNs on important columns
cols_to_show = ['player_id','season','age_at_season_start','total_minutes','goals','target_log1p']
cols_to_show = [c for c in cols_to_show if c in df.columns]
print("\nSample rows:")
display(df[cols_to_show].head(10))

# Show percent of nulls for columns we will operate on
check_cols = list(set(expected_numeric + ['season','player_id','target_log1p']))
print("\nNull percentages (selected):")
for c in check_cols:
    if c in df.columns:
        pct = df[c].isna().mean() * 100
        print(f"  {c}: {pct:.2f}%")


File path: /content/drive/MyDrive/player_value_prediction_project/data/processed/master/final_training_master.csv
Loaded shape: (87223, 109)

Columns (first 80):
 ['player_id', 'season', 'name', 'age_at_season_start', 'position', 'sub_position', 'current_club_id', 'current_club_domestic_competition_id', 'played_any', 'appearances_count', 'total_minutes', 'minutes_per_appearance', 'goals', 'assists', 'goals_per90', 'assists_per90', 'target_eur', 'target_log1p', 'target_choice', 'sb_shots', 'sb_xg', 'sb_goals', 'sb_passes', 'sb_passes_completed', 'sb_tackles', 'sb_interceptions', 'sb_clearances', 'sb_fouls_committed', 'sb_pass_completion', 'sb_xg_per_shot', 'sb_goals_per_shot', 'sb_tackles_per90', 'sb_interceptions_per90', 'sb_clearances_per90', 'sb_fouls_committed_per90', 'reddit_num_posts', 'reddit_num_comments_used', 'reddit_pos_ratio', 'reddit_neu_ratio', 'reddit_neg_ratio', 'reddit_mean_compound', 'reddit_fallback_used', 'reddit_subreddits_covered', 'num_injuries', 'num_unique_injur

,player_id,season,age_at_season_start,total_minutes,goals,target_log1p
0,10,2012,34.0,2585,16,14.508658
1,10,2013,35.0,2220,8,13.815512
2,10,2014,36.0,2289,16,13.815512
3,10,2015,37.0,1714,8,13.815512
4,26,2012,32.0,4401,0,15.424949
5,26,2013,33.0,3855,0,0.000000
6,26,2014,34.0,2880,0,14.914123
7,26,2015,35.0,1260,0,13.815512
8,26,2016,36.0,1020,0,13.815512
9,26,2017,37.0,92,0,13.527830



Null percentages (selected):
  target_log1p: 0.00%
  appearances_count: 0.00%
  age_at_season_start: 0.07%
  transfers_sum_fee: 0.00%
  num_injuries: 0.00%
  reddit_mean_compound: 0.00%
  assists: 0.00%
  player_id: 0.00%
  season: 0.00%
  transfers_count: 0.00%
  total_minutes: 0.00%
  goals: 0.00%
